In [2]:
# 텍스트로 느낌 물어보고 향수추천
import mysql.connector
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
import re

# MySQL 연결 함수
def get_db_connection():
    return mysql.connector.connect(
        host="localhost",
        user="ohgiraffers",
        password="ohgiraffers",
        database="test_db"
    )

# 사용자 느낌에서 키워드 추출하는 함수
def extract_keywords(user_feeling):
    keywords = re.findall(r'\b\w+\b', user_feeling)
    return keywords

# 사용자 느낌에 맞는 향수 데이터를 DB에서 가져오는 함수 (최대 3개)
def get_fragrance_from_db(user_feeling):
    conn = get_db_connection()
    cursor = conn.cursor(dictionary=True)
    
    # 사용자 느낌에서 추출된 키워드 가져오기
    keywords = extract_keywords(user_feeling)
    
    # 키워드로 향수를 찾기 위한 쿼리 (최대 3개)
    query = """
    SELECT name, brand, description
    FROM perfume
    WHERE description LIKE %s
    """
    
    # 키워드마다 LIKE 쿼리를 적용하여 향수 찾기
    perfumes = []
    for keyword in keywords:
        cursor.execute(query, ('%' + keyword + '%',))
        result = cursor.fetchall()
        perfumes.extend(result)
    
    # 중복 제거하고 최대 3개만 가져오기
    perfumes = list({f['name']: f for f in perfumes}.values())[:3]
    
    cursor.close()
    conn.close()
    
    return perfumes

# 하나의 프롬프트를 사용하여 향수 추천 및 이유 설명을 생성하는 함수
def recommend_and_explain_fragrance(user_feeling, perfume_list):
    if not perfume_list:
        return {
            "recommended_perfumes": "데이터베이스에서 조건에 맞는 향수를 찾을 수 없습니다. 다른 느낌을 제공해 주세요.",
            "reason": "",
            "perfume_list": perfume_list
        }
    
    perfume_info = "\n".join(
        [f"이름: {f['name']}, 브랜드: {f['brand']}, 설명: {f['description']}" for f in perfume_list]
    )
    
    prompt_template = """
    사용자가 '{user_feeling}'라는 느낌을 요청했습니다.
    아래는 데이터베이스에서 찾은 향수 목록입니다:
    {perfume_info}
    
    위의 목록에서 사용자의 느낌에 맞는 향수를 추천하고, 선택한 이유를 설명해주세요.
    절대로 목록에 없는 다른 향수를 추천하지 마세요.
    
    응답 형식:
    추천 향수:
    1. [향수 이름] - [브랜드]
    2. [향수 이름] - [브랜드]
    3. [향수 이름] - [브랜드]
    
    추천 이유:
    각각의 향수가 사용자 느낌과 부합하는 이유를 간단히 설명해주세요.
    """

    # LangChain의 PromptTemplate과 ChatOpenAI 사용
    prompt = PromptTemplate.from_template(prompt_template)
    chat_model = ChatOpenAI(model="gpt-4o", temperature=0)
    
    response = chat_model.predict(
        prompt.format(user_feeling=user_feeling, perfume_info=perfume_info)
    )

    return {
        "recommended_perfumes": response,  # 추천 향수 목록 및 이유를 포함한 응답
        "perfume_list": perfume_list
    }

# 사용자 요청에 따른 향수 추천 및 이유 설명 함수
def recommend_fragrance_based_on_feeling(user_feeling):
    perfume = get_fragrance_from_db(user_feeling)
    response = recommend_and_explain_fragrance(user_feeling, perfume)
    
    return response

# 사용자 요청 처리 예시
user_feeling = "결혼식에 어울릴만한 향"
response = recommend_fragrance_based_on_feeling(user_feeling)

# 결과 출력
print("[느낌]:\n" + "-"*40)
print(user_feeling)

print("\n[추천 향수 및 이유]:\n" + "-"*40)
print(response["recommended_perfumes"])

print("\n[데이터베이스 검색 결과]:\n" + "-"*40)
for perfume in response["perfume_list"]:
    print(f"이름: {perfume['name']}, 브랜드: {perfume['brand']}, 설명: {perfume['description']}")
    print("-" * 40)

C:\Users\20118\AppData\Local\Temp\ipykernel_17208\3818239470.py:86: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chat_model.predict(


[느낌]:
----------------------------------------
결혼식에 어울릴만한 향

[추천 향수 및 이유]:
----------------------------------------
추천 향수:
1. 플레르 드 뽀 오 드 퍼퓸 - 딥티크
2. 오드 우드 오 드 퍼퓸 - 톰 포드
3. 테싯 오 드 퍼퓸 - 이솝

추천 이유:
1. 플레르 드 뽀 오 드 퍼퓸 - 딥티크: 이 향수는 로맨틱하고 관능적인 체취를 느낄 수 있는 향으로, 결혼식의 로맨틱한 분위기에 잘 어울립니다.
2. 오드 우드 오 드 퍼퓸 - 톰 포드: 청량한 소나무 계열의 향과 부드러운 침구의 조화가 결혼식의 우아하고 세련된 분위기를 연출하는 데 적합합니다.
3. 테싯 오 드 퍼퓸 - 이솝: 따뜻하고 생기 넘치는 향이 결혼식의 기쁨과 활기를 더해주며, 마음을 릴렉싱 시켜주는 효과로 편안한 분위기를 만들어줍니다.

[데이터베이스 검색 결과]:
----------------------------------------
이름: 오드 우드 오 드 퍼퓸, 브랜드: 톰 포드, 설명: 청량한 소나무 계열의 향과 부드러운 침구가 부드럽게 감싸주는 듯한 향
----------------------------------------
이름: 플레르 드 뽀 오 드 퍼퓸, 브랜드: 딥티크, 설명: 달콤한 숨소리로 로맨틱한 포옹을 하는 듯한 관능적인 체취를 느낄 수 있는 향
----------------------------------------
이름: 테싯 오 드 퍼퓸, 브랜드: 이솝, 설명: 이솝의 시그니처 향기로 따뜻하고 생기넘치며 마음을 릴렉싱 시켜주는 향
----------------------------------------


In [ ]:
# 이미지를 넣으면 어울리는 향수 추천
import requests
from PIL import Image
from io import BytesIO
from transformers import BlipProcessor, BlipForConditionalGeneration
import mysql.connector
from langchain_openai import ChatOpenAI

# MySQL 데이터베이스 연결을 위한 함수
def get_mysql_connection():
    try:
        connection = mysql.connector.connect(
            host="localhost",
            user="ohgiraffers",
            password="ohgiraffers",
            database="test_db"
        )
        return connection
    except mysql.connector.Error as err:
        print(f"DB 연결 오류: {err}")
        return None

# 이미지를 다운로드하고 BLIP 모델을 사용해 캡션을 생성하는 함수
def blip_image_caption(image_url):
    try:
        response = requests.get(image_url)
        if response.status_code == 200:
            image = Image.open(BytesIO(response.content))
        else:
            print(f"이미지 다운로드 실패, 상태 코드: {response.status_code}")
            return "이미지를 다운로드할 수 없습니다."
        
        # BLIP 모델 로드
        processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
        model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

        # 이미지 캡션 생성
        inputs = processor(image, return_tensors="pt")
        out = model.generate(**inputs, max_new_tokens=500)
        caption = processor.decode(out[0], skip_special_tokens=True)
        
        print("이미지 캡션:", caption)
        return caption

    except Exception as e:
        print(f"BLIP 모델 오류: {e}")
        return "이미지를 분석할 수 없습니다."

# 향수 데이터베이스에서 향수 추천을 가져오는 함수
def fetch_perfume_recommendations(caption):
    connection = get_mysql_connection()
    if not connection:
        return []
    cursor = connection.cursor(dictionary=True)
    
    # 최대 5개의 키워드를 추출하여 향수 검색
    keywords = caption.split()[:5]
    like_conditions = " OR ".join(["description LIKE %s"] * len(keywords))
    query = f"""
    SELECT name, brand, description
    FROM perfume
    WHERE {like_conditions}
    LIMIT 3;
    """
    cursor.execute(query, tuple([f"%{keyword}%" for keyword in keywords]))
    results = cursor.fetchall()
    cursor.close()
    connection.close()

    # 향수 결과 출력
    if results:
        print(f"데이터베이스에서 찾은 향수들: {results}")
    else:
        print("향수 결과 없음")

    return results

# GPT를 사용하여 향수 특징과 느낌 생성
def generate_perfume_characteristics_and_feeling_with_gpt(perfumes, user_feeling):
    try:
        llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)
        
        perfume_info = "\n".join([f"{perfume['name']} - {perfume['brand']}: {perfume['description']}" for perfume in perfumes])
        
        prompt_template = f"""
        사용자가 '{user_feeling}'라는 느낌을 요청했습니다.
        아래는 데이터베이스에서 찾은 향수 목록입니다:
        {perfume_info}
        
        위의 목록에서 사용자의 느낌에 맞는 향수를 추천하고, 선택한 이유를 설명해주세요.
        절대로 목록에 없는 다른 향수를 추천하지 마세요.
        
        응답 형식:
        추천 향수:
        1. [향수 이름] - [브랜드]
        2. [향수 이름] - [브랜드]
        3. [향수 이름] - [브랜드]
        
        추천 이유:
        각각의 향수가 사용자 느낌과 부합하는 이유를 간단히 설명해주세요.
        """
        
        response = llm(messages=[ 
            {"role": "system", "content": "당신은 1등급 조향사입니다. 주어진 향수 목록을 분석하고, 사용자가 요청한 느낌에 맞는 향수를 추천하고 이유를 설명해주세요."},
            {"role": "user", "content": prompt_template}
        ])
        
        print("GPT 응답:", response)  # 응답 전체 로그 출력
        
        if response and 'choices' in response and len(response['choices']) > 0:
            content = response['choices'][0].get('message', {}).get('content', "응답 형식이 잘못되었습니다.")
            return content
        else:
            return "GPT 응답 오류: 잘못된 응답 형식"
    
    except Exception as e:
        print(f"GPT 모델 오류: {e}")
        return "향수 분석 오류"
    

# 응답 내용을 분리하는 함수
def recommend_fragrance_based_on_image(image_url):
    caption = blip_image_caption(image_url)
    perfumes = fetch_perfume_recommendations(caption)
    
    if not perfumes:
        return "이미지에서 추출한 느낌에 맞는 향수를 찾을 수 없습니다."
    
    # 결과 출력
    print(f"\n[이미지 설명]: {caption}\n")


# 사용자로부터 이미지 URL을 입력받고 추천 결과 출력
image_url = input("이미지 URL을 입력하세요: ")
recommend_fragrance_based_on_image(image_url)

In [29]:
# 계열찾기
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
import re

# 향 라인 데이터를 직접 정의
FRAGRANCE_LINES = [
    {
        "id": 1,
        "name": "Spicy",
        "content": "자극적이며 섹시한 향",
        "color": "FF5757"
    },
    {
        "id": 2,
        "name": "Chypre",
        "content": "격조있고 성숙한 느낌의 개성있는 향",
        "color": "FF7F43"
    },
    {
        "id": 3,
        "name": "Fruity",
        "content": "달콤하고 귀여운 향",
        "color": "FFBD43"
    },
    {
        "id": 4,
        "name": "Citrus",
        "content": "과일 같은 상큼한 향",
        "color": "FFE043"
    },
    {
        "id": 5,
        "name": "Green",
        "content": "상쾌하고 신선한 향",
        "color": "62D66A"
    },
    {
        "id": 6,
        "name": "Aldehyde",
        "content": "인공 비누향, 모던하고 우아하면 지적인 향",
        "color": "98D1FF"
    },
    {
        "id": 7,
        "name": "Aquatic",
        "content": "들과 바다 시원함이 가득한 향",
        "color": "56D2FF"
    },
    {
        "id": 8,
        "name": "Fougere",
        "content": "부드럽고 싱그러운 남성의 향",
        "color": "7ED3BB"
    },
    {
        "id": 9,
        "name": "Gourmand",
        "content": "달달하고 유혹적인 미각의 즐거움을 느낄 수 있는 향",
        "color": "A1522C"
    },
    {
        "id": 10,
        "name": "Woody",
        "content": "고상하고 증후한 느낌의 따뜻하고 부드러운 향",
        "color": "86390F"
    },
    {
        "id": 11,
        "name": "Oriental",
        "content": "무거우면서 부드럽고 고혹적인 향, 증후하고 달콤하며 센슈얼한 향",
        "color": "C061FF"
    },
    {
        "id": 12,
        "name": "Floral",
        "content": "화려하고 여성스러운 향",
        "color": "FF80C1"
    },
    {
        "id": 13,
        "name": "Musk",
        "content": "이성에게 어필 가능한 아름다운 향",
        "color": "F8E4FF"
    },
    {
        "id": 14,
        "name": "Powdery",
        "content": "포근한 향",
        "color": "FFFFFF"
    },
    {
        "id": 15,
        "name": "Tobacco Leather",
        "content": "마초적인 남성의 개성이 강한 매력적인 향",
        "color": "000000"
    },
    {
        "id": 16,
        "name": "Amber",
        "content": "달콤하고 부드러운 향",
        "color": "FFE8D3"
    }
]
FRAGRANCE_NOTES =[
	{
		"id" : 1,
		"name" : "Bergamot",
		"content" : "citrusy, bitter & tart, elegant, light note with mild spicy tone, complex with nuances of fruit and aromatic elements, reminiscent of eau de Cologne, it flavors Earl Grey tea."
	},
	{
		"id" : 3,
		"name" : "Bitter Orange",
		"content" : "It is inedible tart citrus, known as \"bigarade\". It is intensely scented."
	},
	{
		"id" : 4,
		"name" : "Blood Orange",
		"content" : "It is a mutation of the sweet orange, very sweet and juicy, with a sweet taste and smell."
	},
	{
		"id" : 5,
		"name" : "Buddha's hand",
		"content" : "citrus, lemon, fresh, sweet, green, woody and herbal facetes"
	},
	{
		"id" : 9,
		"name" : "Chinotto",
		"content" : "A citrus note sweet and aromatic, this fruit is used and associated with chinotto beverage."
	},
	{
		"id" : 10,
		"name" : "Citron",
		"content" : "a bright dry citrus note."
	},
	{
		"id" : 12,
		"name" : "Citruses",
		"content" : "A category of hesperidic fruits of different varieties."
	},
	{
		"id" : 13,
		"name" : "Clementine",
		"content" : "a variety of mandarin orange without seeds, known as \"Christmas mandarin\", sweet and juicy."
	},
	{
		"id" : 14,
		"name" : "Finger Lime",
		"content" : "A fresh and sweet, clean citrus fragrance with balsamic coumarin undertones."
	},
	{
		"id" : 15,
		"name" : "Grapefruit",
		"content" : "A bitter aromatic citrus scent with a \"dirty\" undertone reminiscent of black currant."
	},
	{
		"id" : 19,
		"name" : "Hassaku",
		"content" : "Japanese citrus variant, similar in scent profile to mandarine orange."
	},
	{
		"id" : 23,
		"name" : "Lemon",
		"content" : "astringent, sour, sweet undertone, aromatic nuance, bright, cheerful & vibrant."
	},
	{
		"id" : 24,
		"name" : "Lemon Balm",
		"content" : "a minty, lightly spicy aspect, an alternative to lemon."
	},
	{
		"id" : 26,
		"name" : "Lemon Tree",
		"content" : "complex note reproducing the aroma of floral lemon blossoms, crushed green leaves and tart ambience around the living tree."
	},
	{
		"id" : 27,
		"name" : "Lemon Verbena",
		"content" : "a lemon-fresh, clean, floral-citrus, aromatic herbal nuances."
	},
	{
		"id" : 29,
		"name" : "Lemongrass",
		"content" : "an intensely lemon-scented aromatic herb, with spicy aromatic undertones."
	},
	{
		"id" : 31,
		"name" : "Limetta",
		"content" : "A fine citrus fragrance, reminiscent of lime, but less acidic and mildly sweet."
	},
	{
		"id" : 32,
		"name" : "Litsea Cubeba",
		"content" : "a lemon-like, sharp, fresh smell with a sweet undertone."
	},
	{
		"id" : 34,
		"name" : "Mandora",
		"content" : "A sweet aromatic citrus note."
	},
	{
		"id" : 35,
		"name" : "Methyl Pamplemousse",
		"content" : "Fresh herbal, green fruity; citrusy, reminding of grapefruit peel, soapy, woody, green,herbal, floral, violet.\r\n\r\nKey aromachemical in modern colognes, bitter sour top note, fresh and evanescent, grapefruit scent blending with citrus bases. It can also be used as a booster for vetiver accords."
	},
	{
		"id" : 36,
		"name" : "Neroli",
		"content" : "A citrusy, light, slightly bitter note, with hints of orange and honey blossom floral facets, purifying, aromatic, green."
	},
	{
		"id" : 37,
		"name" : "Orange",
		"content" : "A sweet citrus note, somewhat between mandarin and pomelo, a small facet of astringency."
	},
	{
		"id" : 38,
		"name" : "Palestinian Sweet Lime",
		"content" : "Is a low-acid lime used in Palestine for food, juice and rootstock."
	},
	{
		"id" : 42,
		"name" : "Rangpur",
		"content" : "With abundant juice, very acidic, tasty and rich in vitamin C."
	},
	{
		"id" : 45,
		"name" : "Sudachi citrus",
		"content" : "A citrus smell."
	},
	{
		"id" : 46,
		"name" : "Tangelo",
		"content" : "A sweet citrus scent reminiscent of mandarine, but less dry."
	},
	{
		"id" : 47,
		"name" : "Tangerine",
		"content" : "A sweet hesperidic fragrance with a nuance of honey."
	},
	{
		"id" : 49,
		"name" : "Yuzu",
		"content" : "A sweet and very bitter, unique citrus fragrance, with a terpenic nuance."
	},
	{
		"id" : 53,
		"name" : "Almond",
		"content" : "It is gourmand bitter-sweet note, soft and nutty. Bitter almond oil is produced not only from almonds, but cherries, apricots, peaches, and prunes."
	},
	{
		"id" : 61,
		"name" : "Artichoke",
		"content" : "Artichoke has a unique vegetable odor – fleshy, bitter, fatty, green, earthy and slightly sweet and warm."
	},
	{
		"id" : 62,
		"name" : "Banana",
		"content" : "a sweet tropical fruity nuance."
	},
	{
		"id" : 63,
		"name" : "Barberry",
		"content" : "from the species berberis bulgaris, small yellow bloom with sweet scent"
	},
	{
		"id" : 64,
		"name" : "Bearberry",
		"content" : "the prefered fruit of bears, tart red fruits with a tannic taste grown in mountains and even Artic conditions, unsuitable for pregnancy and kidney failure patients."
	},
	{
		"id" : 68,
		"name" : "Black Sapote",
		"content" : "Sweet and nutty, with notes of chocolate."
	},
	{
		"id" : 70,
		"name" : "Blackberry",
		"content" : "Blackberries are a synthetic note with a tangy-sweet fruity profile."
	},
	{
		"id" : 75,
		"name" : "Breadnut",
		"content" : "A savory tropical note."
	},
	{
		"id" : 80,
		"name" : "Carambola (Star Fruit)",
		"content" : "a fresh and sour fruity note, with a hint of citrus and green."
	},
	{
		"id" : 81,
		"name" : "Carrot",
		"content" : "a powdery earthy note (carrot seed), it has iris rhizome characteristics, in fact often substituted for the costlier orris."
	},
	{
		"id" : 85,
		"name" : "Cepes",
		"content" : "an earthy musty appetizing smell of mushrooms."
	},
	{
		"id" : 88,
		"name" : "Cherry",
		"content" : "a popular fruity note, sweet and tart at the same time, with a characteristic bitter almond nuance."
	},
	{
		"id" : 95,
		"name" : "cocoa shell",
		"content" : "A subtle, nutty, earthy chocolate aroma."
	},
	{
		"id" : 96,
		"name" : "Coconut",
		"content" : "A tropical fruity nutty sweet note with vanilla, milky nuances."
	},
	{
		"id" : 104,
		"name" : "Cucumber",
		"content" : "a fresh, aqueous green aroma which diffuses very well thanks to the \"cucumber aldehyde\" (2(E),6(Z)-nonadienal). Violet leaf can yield cucumber, metallic green aspects in perfumes too."
	},
	{
		"id" : 105,
		"name" : "Cupuaçu, Cupuassu, Copoasu",
		"content" : "tropical plant, edible note, related to the tree that produces cocoa; the rind of the fruit smells between chocolate and pineapple while the juice is smelling of pear and green banana."
	},
	{
		"id" : 106,
		"name" : "Currant Leaf and Bud",
		"content" : "leaves and buds of blackcurrants, also known as bourgeons de cassis, have a characteristic green & ammoniac scent reminiscent of \"cat piss\"."
	},
	{
		"id" : 107,
		"name" : "Cyperus Scariosus",
		"content" : "Indian woody note also known as Cypriol or Nagarmotha with facets of vetiver, cedar and patchouli or recreated woody blend to substitute the natural oil."
	},
	{
		"id" : 108,
		"name" : "Daikon Radish",
		"content" : "Daikon is a member of the radish family and shares their characteristic pungent, fresh and mustard-like smell. It is similar to the scent of leafy green kale, mustard oil, with metallic nuances and a sour note similar to dill. It has mild sweetness, a crisp top note like rhubarb, and a softer leafy quality like lettuce."
	},
	{
		"id" : 113,
		"name" : "Dried Fruits",
		"content" : "notes that recall sweet, sticky fruits, such as dates, figs, dried apricot skins and liquorice."
	},
	{
		"id" : 116,
		"name" : "Feijoa Fruit",
		"content" : "It has a pleasant, aromatic and sweet aroma."
	},
	{
		"id" : 117,
		"name" : "Fig",
		"content" : "A complex green sweet lactonic fragrance (leaves). The fruit is very sweet, honeyed, floral, gourmand."
	},
	{
		"id" : 118,
		"name" : "Fig Leaf",
		"content" : "green, bitterish and unique scent that brings to mind earth dust, shadow and foliage"
	},
	{
		"id" : 119,
		"name" : "Forest Fruits",
		"content" : "a wide classification of usually red or blue berries with sweet & sour aspects."
	},
	{
		"id" : 122,
		"name" : "Fruity Notes",
		"content" : "Fresh and sweet fruity notes enhance the beauty and naturalness of floral notes."
	},
	{
		"id" : 123,
		"name" : "Fuji Apple",
		"content" : "A fantasy sweet apple note."
	},
	{
		"id" : 124,
		"name" : "Gariguette Strawberry",
		"content" : "Gariguette Strawberries are the earliest variety of spring strawberries. They have a sweet, tangy flavour and are highly fragrant."
	},
	{
		"id" : 138,
		"name" : "Guava",
		"content" : "a fruity tropical note."
	},
	{
		"id" : 140,
		"name" : "Hog Plum",
		"content" : "Citrus-aromatic, acidic, sunny and long-lasting."
	},
	{
		"id" : 147,
		"name" : "Kiwi",
		"content" : "a tropical sweet and sour fruity note."
	},
	{
		"id" : 150,
		"name" : "Litchi",
		"content" : "Also referenced as \"lychee\", litchi has a tropical sweet fragrance, fruity aquatic."
	},
	{
		"id" : 156,
		"name" : "Malt",
		"content" : "a savory scent reminiscent of roasted barley, beer and whiskey."
	},
	{
		"id" : 157,
		"name" : "Mamey",
		"content" : "It has a sweet, fruity, marzipan odour with hints of cherries and squash."
	},
	{
		"id" : 158,
		"name" : "Mango",
		"content" : "a tropical fruity note, very juicy, with peach-like and piney facets."
	},
	{
		"id" : 159,
		"name" : "Mangosteen",
		"content" : "somewhat sweet and tangy, astringent fruit with wide use in folk medicine (as an anti-inflammatory)."
	},
	{
		"id" : 163,
		"name" : "Melon",
		"content" : "A fresh and sweet fruity aquatic note, usually attributed to Calone, a popular lab synthetic popularized in the 1990s."
	},
	{
		"id" : 164,
		"name" : "Mirabelle",
		"content" : "a fantasy fruity note, reminiscent of mirabelle plum, sweet, fresh, fleshy."
	},
	{
		"id" : 165,
		"name" : "Miracle Berry",
		"content" : "Synsepalum dulcificum is a plant in the Sapotaceae family, native to tropical Africa. It is known for its berry that, when eaten, causes sour foods subsequently consumed to taste sweet."
	},
	{
		"id" : 169,
		"name" : "Mushroom",
		"content" : "an earthy musty note reminiscent of mushrooms."
	},
	{
		"id" : 171,
		"name" : "Nectarine",
		"content" : "a variety of peach, nectarous fruity note with high fructose level and apricot-peachy aroma."
	},
	{
		"id" : 173,
		"name" : "Okra Seeds",
		"content" : "Vegetal musk"
	},
	{
		"id" : 175,
		"name" : "Papaya",
		"content" : "a tropical fruit note, sweet, fleshy and fresh."
	},
	{
		"id" : 176,
		"name" : "Passionfruit",
		"content" : "An intense tropical synthetic fruity note."
	},
	{
		"id" : 178,
		"name" : "Peach",
		"content" : "A very popular fruity note, fleshy, sweet, nectarous and powdery."
	},
	{
		"id" : 180,
		"name" : "Pear",
		"content" : "A sweet and fresh, aquatic fruity note."
	},
	{
		"id" : 181,
		"name" : "Pecan",
		"content" : "A fantasy gourmand note with a nutty, sweet, woody profile."
	},
	{
		"id" : 184,
		"name" : "Pineapple",
		"content" : "A fresh and juicy fruity note, fresh and slightly metallic."
	},
	{
		"id" : 186,
		"name" : "Pistachio",
		"content" : "a green nutty savory note."
	},
	{
		"id" : 187,
		"name" : "Pitahaya",
		"content" : "a fruit of cactus species, known as \"dragon fruit\", native to the Americas. Known also as \"moonflower\", the flowers exude a sweet, heady floral aroma when in bloom."
	},
	{
		"id" : 189,
		"name" : "Plum",
		"content" : "A sensual fruity fragrance."
	},
	{
		"id" : 190,
		"name" : "Pomegranate",
		"content" : "A fantasy fruity note, sweet and sour, spicy and leathery."
	},
	{
		"id" : 193,
		"name" : "Pumpkin",
		"content" : "soft, carbohydrate rich flavor, pulpous note reminiscent of pumpkin pie and American baking."
	},
	{
		"id" : 198,
		"name" : "Red Apple",
		"content" : "It is a fresh and sweet synthetic fruity note."
	},
	{
		"id" : 200,
		"name" : "Red Currant",
		"content" : "a sour aromatic fruity note."
	},
	{
		"id" : 204,
		"name" : "Rhubarb",
		"content" : "a sweet and sour, green and mouthwatering, vegetal spicy fruity fragrance."
	},
	{
		"id" : 216,
		"name" : "Soybean",
		"content" : "legume note with a fermented bean paste flavor."
	},
	{
		"id" : 219,
		"name" : "Strawberry",
		"content" : "A sweet fruity note, artificial or composed of other materials, natural or artificial."
	},
	{
		"id" : 221,
		"name" : "Tayberry",
		"content" : "another Rubus genus variant, cross between blackberry and red raspberry, with fruit larger and sweeter than the loganberry (also a hybrid)."
	},
	{
		"id" : 222,
		"name" : "Tomato",
		"content" : "a penetrating intense aromatic scent of tomato leaves."
	},
	{
		"id" : 223,
		"name" : "Tropical Fruits",
		"content" : "a general scent of sweet, lush, mouthwatering tropical fruits: coconut, pineapple, guava, etc."
	},
	{
		"id" : 224,
		"name" : "Tropicalone",
		"content" : "Exotic tropical fruits such as yellow passion fruit (lilikoi), pineapple, red fruits, mangosteen, and pitahaya with natural creamy/milky nuances"
	},
	{
		"id" : 229,
		"name" : "Watermelon",
		"content" : "an extremely fresh note related to Calone or similar in profile syntethic molecules."
	},
	{
		"id" : 231,
		"name" : "White Currant",
		"content" : "part of the Ribes genus, translucent berries with sweeter flavor than red currants."
	},
	{
		"id" : 233,
		"name" : "Wild Strawberry",
		"content" : "a rich foresty sweet fruity fragrance."
	},
	{
		"id" : 237,
		"name" : "Yellow Cherry",
		"content" : "a cultivar known as Rainier cherry, sweet and with creamy-yellow flesh."
	},
	{
		"id" : 243,
		"name" : "Aglaia",
		"content" : "A fresh and citrusy floral note of great radiance."
	},
	{
		"id" : 248,
		"name" : "Althaea",
		"content" : "Roots of althea or marshmallow plant used as a vegetal source of musk."
	},
	{
		"id" : 250,
		"name" : "Alyssum",
		"content" : "A honey sweet floral note."
	},
	{
		"id" : 251,
		"name" : "Amarillys",
		"content" : "a floral note with fruity undertone between nectarine and rose."
	},
	{
		"id" : 253,
		"name" : "Anemone",
		"content" : "soft lightly powdery and pungent scent of real anemones (which smell especially pungent when dead)"
	},
	{
		"id" : 254,
		"name" : "Angel's Trumpet",
		"content" : "a very fragrant variation of datura plant, sweet, penetrating, intoxicating, tropical night flower scent"
	},
	{
		"id" : 255,
		"name" : "Apple Blossom",
		"content" : "a fantasy fruity floral note, light and spring-like."
	},
	{
		"id" : 256,
		"name" : "Apricot Blossom",
		"content" : "a fantasy floral note of a delicate airy character."
	},
	{
		"id" : 268,
		"name" : "Black Currant Blossom",
		"content" : "sweet, fruity scent, less \"catty\" than the leaves and buds, with a tangy background nuance"
	},
	{
		"id" : 271,
		"name" : "Blue Poppy",
		"content" : "fantasy note inspired by the Himmalayan Blue Poppy which oddly has no fragrance!"
	},
	{
		"id" : 272,
		"name" : "Bluebell",
		"content" : "Clear, spring-like, sweet, green, delicate floral."
	},
	{
		"id" : 282,
		"name" : "Camellia",
		"content" : "a fantasy floral note based on the idea of camellia, a beautiful scentless flower"
	},
	{
		"id" : 291,
		"name" : "Chamomile",
		"content" : "An aromatic herbal fragrance with green and spicy nuances."
	},
	{
		"id" : 292,
		"name" : "Champaca",
		"content" : "A warm heady floral."
	},
	{
		"id" : 293,
		"name" : "Cherry Blossom",
		"content" : "A fantasy floral note of light fresh blooms."
	},
	{
		"id" : 297,
		"name" : "Chocolate Flower",
		"content" : "smelling of chocolate, delicious, bittersweet note"
	},
	{
		"id" : 299,
		"name" : "Chrysanthemum",
		"content" : "an herbal, slightly musty note with green and spicy facets."
	},
	{
		"id" : 301,
		"name" : "Clematis",
		"content" : "Different species of clematis offer different fragrance profiles, from minty-chocolate to honeyed."
	},
	{
		"id" : 302,
		"name" : "Clover",
		"content" : "a sweet honey floral and green note."
	},
	{
		"id" : 307,
		"name" : "Costus",
		"content" : "a sensual and dirty, rooty note reminiscent of unwashed hair."
	},
	{
		"id" : 312,
		"name" : "Cyclamen",
		"content" : "A fine synthetic note replicating the refined, light, clean and mysterious scent of cyclamen flowers."
	},
	{
		"id" : 315,
		"name" : "Dandelion",
		"content" : "citrusy, hint of rose, aromatic, bitter-sweet bouquet."
	},
	{
		"id" : 318,
		"name" : "Deadnettle",
		"content" : "Deadnettle essential oil contains a high amount of Germacrene D. It possesses a complex woody piney hay earthy green scent."
	},
	{
		"id" : 328,
		"name" : "Elderflower",
		"content" : "part of the Sambucus family with a floral-herbal aroma and intense berry scent when infused"
	},
	{
		"id" : 330,
		"name" : "English Marigold",
		"content" : "an herbal, aromatic fragrance."
	},
	{
		"id" : 339,
		"name" : "Flax",
		"content" : "a mild, nutty scent from flax seeds, which can turn fishy when rancid"
	},
	{
		"id" : 340,
		"name" : "Floral Notes",
		"content" : "Floral notes are an essential part of almost every perfume. They can be natural or synthetic."
	},
	{
		"id" : 341,
		"name" : "Forget Me Not",
		"content" : "A fantasy note with transparent green fresh nuances."
	},
	{
		"id" : 343,
		"name" : "Freesia",
		"content" : "A fresh spicy powdery floral note."
	},
	{
		"id" : 351,
		"name" : "Grapeflower",
		"content" : "A fantasy floral accord created of natural and synthetic materials."
	},
	{
		"id" : 359,
		"name" : "Hawthorn",
		"content" : "a strong floral honeyed sweet fragrance with dirty animalic nuances."
	},
	{
		"id" : 361,
		"name" : "Heather",
		"content" : "a sweet and aromatic honey floral note"
	},
	{
		"id" : 362,
		"name" : "Heliotrope",
		"content" : "almond-smelling, vanilla-like note with very hazy, flou contours, recreated via heliotropin"
	},
	{
		"id" : 363,
		"name" : "Hellabore Flower",
		"content" : "powdery smelling note that recalls primroses"
	},
	{
		"id" : 372,
		"name" : "Hoya Carnosa Wax Plant",
		"content" : "The scent depends on certain specie of the plant, can vary from vanilla, sweet to fresh citrusy."
	},
	{
		"id" : 373,
		"name" : "Hyacinth",
		"content" : "An intense sharp green multifaceted floral note."
	},
	{
		"id" : 374,
		"name" : "Hyssop",
		"content" : "Hyssop oil has a warm aromatic sharp camphoraceous scent."
	},
	{
		"id" : 375,
		"name" : "Impatiens",
		"content" : "A fantasy delicate floral note of fragile shade loving impatiens."
	},
	{
		"id" : 377,
		"name" : "Iris",
		"content" : "A natural iris (iris root) note is earthy, woody, powdery, reminiscent of boiled carrot. A fantasy iris note is a powdery floral, reminiscent of the violet flower."
	},
	{
		"id" : 382,
		"name" : "Jarana Flower",
		"content" : "Vibrant watery and intense floral touch. Its smell is reminiscent of the fresh waters of Brazilian rivers."
	},
	{
		"id" : 385,
		"name" : "Kadam",
		"content" : "A sweet tropical floral woody scent, similar to champaca."
	},
	{
		"id" : 389,
		"name" : "Kiwi blossom",
		"content" : "A tropical fruity floral tropical note."
	},
	{
		"id" : 391,
		"name" : "Laburnum",
		"content" : "also called \"golden chain tree\", lightly fragrant tree with resinous properties of its sapwood for musical instruments use"
	},
	{
		"id" : 399,
		"name" : "Lavender",
		"content" : "An aromatic floral clean note, with green, fresh spicy, licorice facets."
	},
	{
		"id" : 402,
		"name" : "Liatris",
		"content" : "a sweet floral, sultry, honey note."
	},
	{
		"id" : 404,
		"name" : "Lilac",
		"content" : "A fine floral powdery fragrance with nuances of honey and jasmine-like green freshness."
	},
	{
		"id" : 411,
		"name" : "Lydia Broom",
		"content" : "plant belonging in the Genista family with dry, bracken-like and honey nuanced scent profile"
	},
	{
		"id" : 414,
		"name" : "Magnolia",
		"content" : "waxy lemony tinged floral, fresh and lush at the same time"
	},
	{
		"id" : 415,
		"name" : "Magnolia Brooklynensis",
		"content" : "A fantasy floral note"
	},
	{
		"id" : 416,
		"name" : "Magnolia Leaf",
		"content" : "Leaves of white champaca have a sweet green gentle fragrance when crushed."
	},
	{
		"id" : 418,
		"name" : "Malva",
		"content" : "The scent is rosy, aromatic, spicy, green."
	},
	{
		"id" : 419,
		"name" : "Mango Blossom",
		"content" : "a very sweet floral scent with fruity tropical nuance"
	},
	{
		"id" : 426,
		"name" : "Mignonette",
		"content" : "a very sweet-smelling and pleasant Mediterranean flower with violet-like and fruity nuances."
	},
	{
		"id" : 427,
		"name" : "Mimosa",
		"content" : "A very sweet powdery floral note with light green and woody nuances."
	},
	{
		"id" : 429,
		"name" : "Mirabilis",
		"content" : "also called 4 o'clock flower or marvel of Peru, fantasy note reminiscent of the decorative beauty of the small bell shaped flowers which smell intensely sweet"
	},
	{
		"id" : 431,
		"name" : "Monoi Oil",
		"content" : "a note of tiare flower mixed with coconut, usually associated with suntan lotion."
	},
	{
		"id" : 433,
		"name" : "Moss Flox",
		"content" : "The smell of its flowers somewhat resembles the one of marijuana."
	},
	{
		"id" : 434,
		"name" : "Myrtle",
		"content" : "a herbal camphoraceous fragrance with sweet and fresh aromatic nuances."
	},
	{
		"id" : 435,
		"name" : "Narcissus",
		"content" : "It has a heady, opulent floral animalic scent with green nuances and a hint of hay and tobacco."
	},
	{
		"id" : 437,
		"name" : "Nasturtium",
		"content" : "a sweet green and spicy floral note."
	},
	{
		"id" : 439,
		"name" : "Nerium Oleander",
		"content" : "a talcum-like floral note, with hints of pollen sweetness"
	},
	{
		"id" : 440,
		"name" : "Nigella",
		"content" : "The nigella seeds oil, after aging, has a pleasant fruity-woody musky fragrance, like ambrette seeds oil, cognac, and wild strawberries."
	},
	{
		"id" : 441,
		"name" : "Night Blooming Jasmine",
		"content" : "an intense sweet jasmine-like fragrance."
	},
	{
		"id" : 449,
		"name" : "Orchid",
		"content" : "Usually a fantasy synthetic floral note."
	},
	{
		"id" : 454,
		"name" : "Orris Root",
		"content" : "A powdery earthy rooty scent, with woody, violet flower nuances."
	},
	{
		"id" : 459,
		"name" : "Passion Flower",
		"content" : "A sweet tropical fragrance, reminiscent of grapes."
	},
	{
		"id" : 462,
		"name" : "Pear Blossom",
		"content" : "An airy fruity floral note."
	},
	{
		"id" : 464,
		"name" : "Peony",
		"content" : "A fresh floral fantasy note. It can't be extracted from peony flowers."
	},
	{
		"id" : 466,
		"name" : "Petalia",
		"content" : "It has a fresh floral scent of peony, lily-of-the-valley, rose with a fruity undertone. Very fresh, light and diffusive."
	},
	{
		"id" : 469,
		"name" : "Pineapple Blossom",
		"content" : "A fantasy fruity floral note."
	},
	{
		"id" : 472,
		"name" : "Pinwheel Flower",
		"content" : "Pinwheel jasmines are found in tropical regions all over the world and are also grown indoors. They have ceremonial importance to certain indigenous peoples of South America. The common name for these shrubs and trees derives from the milky sap that the plants exude. Some wood sap serves as food for the larvae of certain moths. They are sweet and have an intense white floral color."
	},
	{
		"id" : 477,
		"name" : "Poinsettia",
		"content" : "flower with a pine-like terpenic scent profile, prone to fungus though giving it a mushroom note"
	},
	{
		"id" : 478,
		"name" : "Pomegranate Blossom",
		"content" : "sweetish tangy bloom with a nuance of dark fruit"
	},
	{
		"id" : 482,
		"name" : "Privet",
		"content" : "spring-smelling, fresh note reminiscent of the privet bush with a cat-piss nuance sometimes"
	},
	{
		"id" : 486,
		"name" : "Rangoon Creeper",
		"content" : "Sweet and fruity, strong and intoxicating, reminiscent of white flowers and honeysuckle."
	},
	{
		"id" : 487,
		"name" : "Raspberry Blossom",
		"content" : "A fantasy fruity floral note with a nuance of rose."
	},
	{
		"id" : 492,
		"name" : "Rosa Alba",
		"content" : "With an elegant and very floral aroma, it manages to be subtle with a sweet background."
	},
	{
		"id" : 494,
		"name" : "Rose",
		"content" : "the king of flowers, lemony fresh with various nuances of powder, wood notes or fruit, feminine, clean, intensely romantic"
	},
	{
		"id" : 495,
		"name" : "Rose Hip",
		"content" : "a rosy fruity note."
	},
	{
		"id" : 496,
		"name" : "Rose Japanese (Hamanasu)",
		"content" : "orange-laced note with sweet aspects"
	},
	{
		"id" : 497,
		"name" : "Rose Mallow",
		"content" : "flower with a fresh aniseed undertone, used in perfumery paired with resins and intense, heady florals such as narcissus"
	},
	{
		"id" : 501,
		"name" : "Rosyfolia",
		"content" : "Rosyfolia® is a floral, rosy, geraniol note with an elegant, fruity airiness."
	},
	{
		"id" : 502,
		"name" : "Safflower",
		"content" : "A delicate floral fragrance."
	},
	{
		"id" : 505,
		"name" : "Sandalwood Flower",
		"content" : "Flowers of Santalum album have a pleasant sandalwood smell."
	},
	{
		"id" : 509,
		"name" : "Siberian Rhododendron",
		"content" : "A hespiridic floral with aromatic spicy amber undertone."
	},
	{
		"id" : 515,
		"name" : "Snowdrops",
		"content" : "a spring-smelling, cool, fresh fantasy floral note inspired by the tiny blossoms of snowdrop."
	},
	{
		"id" : 518,
		"name" : "Sour Cherry Blossom",
		"content" : "a fantasy floral fruity note of a delicate nature."
	},
	{
		"id" : 519,
		"name" : "Spanish Broom",
		"content" : "A sweet yellow floral with honey, pollen nuances."
	},
	{
		"id" : 524,
		"name" : "Strelitzia",
		"content" : "also called Bird of Paradise, subtly tropical and herbal smelling, a fantasy note in perfumes"
	},
	{
		"id" : 526,
		"name" : "Sunflower",
		"content" : "a subtle scent of herbal and hay-like nuance, very soft."
	},
	{
		"id" : 527,
		"name" : "Sweet Pea",
		"content" : "a light sweet fruity floral note."
	},
	{
		"id" : 528,
		"name" : "Taif Rose",
		"content" : "a variety of rose that has a faint tea-like tannic nuance to its powdery soft bouquet"
	},
	{
		"id" : 530,
		"name" : "Tobacco Blossom",
		"content" : "summary hay and leather scent with dry floral tonality, very inviting, and not at all smelling like rich pipe tobacco or cigarette ash"
	},
	{
		"id" : 535,
		"name" : "Viburnum",
		"content" : "Viburnum flowers are delicately sweetly scented with a hint of vanilla, almond and fruity nuances, similar to heliotrope and honeysuckle."
	},
	{
		"id" : 541,
		"name" : "Water Hyacinth",
		"content" : "a green floral with an aquatic nuance, less dramatic than the regular hyacinth."
	},
	{
		"id" : 542,
		"name" : "Water Lily",
		"content" : "A fresh and fine soft aquatic floral fragrance."
	},
	{
		"id" : 548,
		"name" : "Wildflowers",
		"content" : "a fantasy note of aromatic and sweet wild flowers with an herbal nuance."
	},
	{
		"id" : 549,
		"name" : "Wisteria",
		"content" : "a spicy floral note with nuances of lilac and clove."
	},
	{
		"id" : 556,
		"name" : "Arum Lily",
		"content" : "a mild scent of pollen and waxy petals."
	},
	{
		"id" : 558,
		"name" : "Black Locust",
		"content" : "an intense and pleasant floral note reminiscent of mimosa and orange blossom."
	},
	{
		"id" : 559,
		"name" : "Boronia",
		"content" : "an airy fresh and fruity sweet floral note."
	},
	{
		"id" : 560,
		"name" : "Carissa",
		"content" : "Similar to jasmine and gardenia, sweet white floral fragrance with a fruity nuance."
	},
	{
		"id" : 561,
		"name" : "Datura",
		"content" : "a fantasy note of very sweet narcotic scent. The flower and all parts of the plant are extremely poisonous."
	},
	{
		"id" : 562,
		"name" : "Frangipani",
		"content" : "A tropical floral fruity note with a citrus nuance."
	},
	{
		"id" : 563,
		"name" : "Gardenia",
		"content" : "An intoxicating white floral note with piquant green and earthy mushroom nuances."
	},
	{
		"id" : 564,
		"name" : "Grapefruit Blossom",
		"content" : "A white floral scent reminiscent of orange blossom, but fresher, less animalic."
	},
	{
		"id" : 565,
		"name" : "Honeysuckle",
		"content" : "A sweet floral note with a nuance of honey and pollen."
	},
	{
		"id" : 566,
		"name" : "Jasmine",
		"content" : "Sweet, white floral, opulent, can be more indolic or greener and fresher when synthesized in the lab."
	},
	{
		"id" : 568,
		"name" : "Lemon Blossom",
		"content" : "a lighter, airier and tangier sparkling floral note than orange blossom, less animalic and sweet."
	},
	{
		"id" : 569,
		"name" : "Lily",
		"content" : "A spicy floral note, sweet and waxy."
	},
	{
		"id" : 570,
		"name" : "Mandarin Orange Blossom",
		"content" : "a sweet floral and somewhat fruity-smelling note close to orange blossom"
	},
	{
		"id" : 572,
		"name" : "Mock Orange",
		"content" : "a floral note of Philadelphus Coronarius (Absolu de Seringe), it is often confused with lilac, but it is not even related. The note has a sweet rich floral profile reminiscent of gardenia, honeysuckle, orange blossom."
	},
	{
		"id" : 574,
		"name" : "Night Blooming Cereus",
		"content" : "It is a tropical cactus plant with flowers that have a dusty white floral smell with a soft spicy undertone."
	},
	{
		"id" : 575,
		"name" : "Orange Blossom",
		"content" : "A fresh and sweet white floral note with an animalic undertone."
	},
	{
		"id" : 577,
		"name" : "Syringa",
		"content" : "a floral note of Philadelphus Coronarius (Absolu de Seringe), it is often confused with lilac, but it is not even related. The note has a sweet rich floral profile reminiscent of gardenia, honeysuckle, orange blossom."
	},
	{
		"id" : 578,
		"name" : "Tangerine Blossom",
		"content" : "smells like mandarin orange blossom"
	},
	{
		"id" : 579,
		"name" : "Tiare Flower",
		"content" : "A white floral note, creamy, fruity and indolic."
	},
	{
		"id" : 585,
		"name" : "Aloe Vera",
		"content" : "a green sappy note."
	},
	{
		"id" : 587,
		"name" : "Angelica",
		"content" : "a musky woody, sweet rooty smell."
	},
	{
		"id" : 590,
		"name" : "Arnica",
		"content" : "grassy and dusty smelling flowers, the plant is used for its medicinal uses"
	},
	{
		"id" : 595,
		"name" : "Avocado",
		"content" : "oily, green vegetal scent, not frequently used in perfumery"
	},
	{
		"id" : 598,
		"name" : "Banana Leaf",
		"content" : "green and sappy, fresh note."
	},
	{
		"id" : 612,
		"name" : "Buchu or Agathosma",
		"content" : "The essential oil is steam distilled from dried leaves of Barosma Betulina. It has a peculiar penetrating bitter-sweet, camphor, phenolic."
	},
	{
		"id" : 615,
		"name" : "Bulrush",
		"content" : "unusual herbal green aqueous note, very rare in perfumery"
	},
	{
		"id" : 618,
		"name" : "Calamus",
		"content" : "a warm spicy woody note with sweet undertones."
	},
	{
		"id" : 619,
		"name" : "Calycanthus",
		"content" : "known as Spice Bush (calycanthus occidentalis) or Sweet Shrub (calycanthus floridus), it smells like Burgundy Wine or fruit (grapefruit and strawberry) according to variety"
	},
	{
		"id" : 621,
		"name" : "Cannabis",
		"content" : "an aromatic smoky note with a dirty sour nuance."
	},
	{
		"id" : 627,
		"name" : "Celery",
		"content" : "Celery oil is extracted from seeds of the plant. It has a familiar celery smell, but less green and fresh."
	},
	{
		"id" : 628,
		"name" : "Celery Seeds",
		"content" : "seeds that smell like celery, warm and earthy, but less green and fresh."
	},
	{
		"id" : 638,
		"name" : "Coleus",
		"content" : "Earthy, woody, smoky, aromatic, clay."
	},
	{
		"id" : 640,
		"name" : "Crithmum",
		"content" : "An aromatic warm spicy note."
	},
	{
		"id" : 641,
		"name" : "Davana",
		"content" : "a sharp herbal bitter green note with a balsamic undertone."
	},
	{
		"id" : 646,
		"name" : "Fern",
		"content" : "a fantasy note reminiscent of a shadowy forest's flora, often used as a term for the fougere family of fragrances"
	},
	{
		"id" : 647,
		"name" : "Flouve",
		"content" : "The Flouve herb, originally from France, has a bittersweet, complex and charming appearance."
	},
	{
		"id" : 659,
		"name" : "Grass",
		"content" : "a sweet yet very fresh scent of cut grass, green uplifting."
	},
	{
		"id" : 663,
		"name" : "Green Notes",
		"content" : "A generic term for notes that evoke snapped leaves, foliage, green vegetal scents."
	},
	{
		"id" : 668,
		"name" : "Hat Straw",
		"content" : "A fantasy note of a straw hat."
	},
	{
		"id" : 673,
		"name" : "Immortelle",
		"content" : "A sweet and dry herbaceous note, reminiscent of hay, tobacco, and honey."
	},
	{
		"id" : 676,
		"name" : "Jatamansi or Spikenard",
		"content" : "An earthy aromatic herbal note."
	},
	{
		"id" : 678,
		"name" : "Juniper",
		"content" : "It has a fresh balsamic, aromatic fragrance with piney nuances."
	},
	{
		"id" : 680,
		"name" : "Katsura Leaf",
		"content" : "In fall, dying leaves of Katsura tree give off a sweet caramel-like fragrance due to maltol occured in the leaves."
	},
	{
		"id" : 682,
		"name" : "Kunzea",
		"content" : "A fresh and sweet aromatic eucalyptus-like scent."
	},
	{
		"id" : 686,
		"name" : "Laurissilva Forest",
		"content" : "Laurissilva is a type of very humid forest, composed mostly of trees from the Lauraceae family and endemic to Macaronesia, a region formed by the archipelagos of Madeira, Azores, Canary Islands and Cape Verde."
	},
	{
		"id" : 691,
		"name" : "Linaloe Berry",
		"content" : "It has a light aromatic citrus scent."
	},
	{
		"id" : 696,
		"name" : "Marjoram",
		"content" : "a spicy herbal dry aromatic note."
	},
	{
		"id" : 698,
		"name" : "Mate",
		"content" : "a bitterish tea note from Latin America, sometimes used to reinforce herbal aspects or to tone down florals."
	},
	{
		"id" : 701,
		"name" : "Mint",
		"content" : "A refreshing and cooling green note with peppery aspects."
	},
	{
		"id" : 702,
		"name" : "Mistletoe",
		"content" : "evergreen note characteristic to Christmas and rarely used in perfumery"
	},
	{
		"id" : 709,
		"name" : "Olive Leaf",
		"content" : "Comes from the olive tree, one of the oldest trees known to man, which is sacred in the Mediterranean basin. The leaf has a bitterish green scent that is viscous and smoky. A branch of olive leaves is considered the symbol of peace."
	},
	{
		"id" : 713,
		"name" : "Palm Leaf",
		"content" : "a vegetal green scent."
	},
	{
		"id" : 714,
		"name" : "Palmarosa",
		"content" : "unusual rosy green soft note with vegetal nuances"
	},
	{
		"id" : 715,
		"name" : "Pandanus",
		"content" : "it has a very specific sharp and powerful, but also sweet, honey-like diffusive fragrance."
	},
	{
		"id" : 718,
		"name" : "Pear Leaf",
		"content" : "green note with waxy nuances, mild"
	},
	{
		"id" : 719,
		"name" : "Pennyroyal",
		"content" : "With a spicy and herbaceous aromatic content."
	},
	{
		"id" : 721,
		"name" : "Petrichor",
		"content" : "Earthy aroma produced by rain when it starts to fall and hits the ground, especially if the weather is hot or dry."
	},
	{
		"id" : 722,
		"name" : "Peyote",
		"content" : "A fantasy green note."
	},
	{
		"id" : 730,
		"name" : "Rice",
		"content" : "a fantasy note, grassy, nutty and earthy, reminiscent of steamed basmati rice, warm and cozy."
	},
	{
		"id" : 731,
		"name" : "Rooibos Red Tea",
		"content" : "a sweetish herbal infusion that is not really tea (i.e. Camelia sinensis), coming from the Aspalathus linear plant instead"
	},
	{
		"id" : 735,
		"name" : "Rosemary",
		"content" : "Аn aromatic culinary herb with a camphorous, minty profile."
	},
	{
		"id" : 737,
		"name" : "Rue",
		"content" : "A sharp herbaceous note with sweet fruity undertones."
	},
	{
		"id" : 741,
		"name" : "Sage",
		"content" : "savory, aromatic culinary note with hazy, soft and mildly peppery aspects belonging to the salvia family"
	},
	{
		"id" : 746,
		"name" : "Seaweed",
		"content" : "a salty, marine, iodine, aromatic fragrance."
	},
	{
		"id" : 747,
		"name" : "Senecio",
		"content" : "With an oily, green, strong and giant aroma."
	},
	{
		"id" : 748,
		"name" : "Shiso",
		"content" : "a member of the mint family, Asian culinary herb which has a peppery fresh green profile"
	},
	{
		"id" : 751,
		"name" : "Siriuba Tree",
		"content" : "The tree has specialized structures called \"salt glands\", which are so called because they excrete salt that is absorbed by the plant. Woody and saline odor."
	},
	{
		"id" : 752,
		"name" : "Skunk Cabbage",
		"content" : "a musky note of vegetal origin"
	},
	{
		"id" : 754,
		"name" : "Spearmint",
		"content" : "A refreshing and cooling green mint note with a dash of sweetness and mellow spiciness."
	},
	{
		"id" : 757,
		"name" : "Strawberry Leaf",
		"content" : "tart green scent of snapped leaves"
	},
	{
		"id" : 761,
		"name" : "Sweet Grass",
		"content" : "A sweet grassy coumarin scent."
	},
	{
		"id" : 762,
		"name" : "Tangerine Leaves",
		"content" : "Green, oily, explosive and earthy citrus odor."
	},
	{
		"id" : 763,
		"name" : "Tansy",
		"content" : "A sharp and spicy bitter herbaceous fragrance, getting more pungent in higher concentrations."
	},
	{
		"id" : 764,
		"name" : "Tarragon",
		"content" : "a herbaceous sweet, mildly anisic note."
	},
	{
		"id" : 765,
		"name" : "Tea",
		"content" : "A sweet herbaceous green fragrance."
	},
	{
		"id" : 767,
		"name" : "Thyme",
		"content" : "Thyme has a strong rich herbaceous fragrance. Its quality depends on the method of extraction."
	},
	{
		"id" : 771,
		"name" : "Tomato Leaf",
		"content" : "a green sharp aromatic note."
	},
	{
		"id" : 774,
		"name" : "Tulsi",
		"content" : "grassy, herbal plant note with ties with basil and comparable scent"
	},
	{
		"id" : 781,
		"name" : "Wheat",
		"content" : "a cereal-like note with a nutty aspect, used to render a soft, warm, attractive note in oriental/gourmand compositions or whenever a mild modifier is needed."
	},
	{
		"id" : 787,
		"name" : "Wormwood",
		"content" : "A bitterish, herbal and strong-smelling plant note known as wormwood which aromatizes absinthe and vermouth."
	},
	{
		"id" : 791,
		"name" : "Anise",
		"content" : "the scent of aniseed, rich in anethol, reminiscent of licorice. Belongs in the same family of scents as tarragon and fennel (also rich in anethol). Very popular in fragrances, notably L'Heure Bleu."
	},
	{
		"id" : 792,
		"name" : "Asafoetida",
		"content" : "the dried oleoresin of the taproot of the herb known as Ferula with an odor profile between leek and onion. Also known as Devil's Dung, Hing, Ting or stinking gum. Ferula is featured in classics, such as Ma Griffe, and modern fragrances, such as Premier Figuier, alike."
	},
	{
		"id" : 793,
		"name" : "Bay Leaf",
		"content" : "an aromatic camphorous fragrance."
	},
	{
		"id" : 795,
		"name" : "Cacao Pod",
		"content" : "A warm spicy gourmand fragrance with amber, balsamic nuances."
	},
	{
		"id" : 796,
		"name" : "Caraway",
		"content" : "a sharp and sweet, lightly spicy note coming from the essential oil of caraway seeds when steam distilled."
	},
	{
		"id" : 797,
		"name" : "Cardamom",
		"content" : "A sweet resinous aromatic and spicy note with a bitter aftertaste."
	},
	{
		"id" : 799,
		"name" : "Cassia",
		"content" : "cinnamon-like, spicy, earthy, aromatic facets, coming from Cinnamomum aromaticum. Very concentrated, intense spice note."
	},
	{
		"id" : 800,
		"name" : "Chutney",
		"content" : "a fantasy note inspired by the recipe of India cuisine and accompanying Far East dishes relying on a mix of spices, fruits and vegetables. Sweet, pungent, hot or cold in feel according to the effect desired."
	},
	{
		"id" : 801,
		"name" : "Cinnamon",
		"content" : "A sweet and warm, powdery tenacious spicy note."
	},
	{
		"id" : 803,
		"name" : "Clove Leaf",
		"content" : "The smell of clove leaf oil is milder than the clove oil, it is used as a replacement to higher-priced oils, like, for example, black pepper. It is also a source for some other perfume materials extracted from it."
	},
	{
		"id" : 805,
		"name" : "Coffee",
		"content" : "Coffee berries are produced by the evergreen bush of the genus Coffea. In perfumes the coffee note is an intense, dark roast facet that is linked to the cocoa facet of patchouli in Borneo 1834 (Lutens). Famously put in good use in A*men by Mugler, L'Or de Torrente"
	},
	{
		"id" : 810,
		"name" : "Cumin",
		"content" : "a spice with an especially pungent, bitter-sour note that can resemble sweat. Polarizing, it can highlight a fragrance creation like no other and is nowadays often used to render an intimate, animalic note in abscence of animalics. Famously overdosed in Kingdom by Alexander McQueen."
	},
	{
		"id" : 813,
		"name" : "Curry Tree",
		"content" : "Also known as Murraya koenigii and \"sweet neem leaves\", this is a spicy aromatic note reminiscent of the leaves and seeds of the Indian peninsula growing tree. Featured in Oriental Lounge by The Different Company."
	},
	{
		"id" : 814,
		"name" : "Dill",
		"content" : "a culinary herb rich in anethole with a delicate herbaceous and slightly spicy scent. Pairs well with other spicy herbs such as basil, tarragon, fennel or licorice in aromatic and oriental compositions."
	},
	{
		"id" : 816,
		"name" : "Fenugreek",
		"content" : "the essence from the yellowish-golden seeds of Trigonella Foenum Graecum is used as a spice. It renders a maple-syrup note in fragrance, common also to helichrysum/immortelle blossom. The note is recognizable (via immortelle essence) in Sables by A.Goutal and (via fenugreek spice) in Santal Blush by Tom Ford."
	},
	{
		"id" : 817,
		"name" : "Galanga",
		"content" : "a spicy balsamic pleasant fragrance. The oil is produced from rhizomes of Alpinia Officinarum."
	},
	{
		"id" : 818,
		"name" : "Ginger",
		"content" : "A spicy fizzy top note, with a pronounced fresh citrus facet and piney nuance."
	},
	{
		"id" : 819,
		"name" : "Green Coffee",
		"content" : "Slightly sweet and neutral."
	},
	{
		"id" : 821,
		"name" : "Indian Spices",
		"content" : "Umbrella term used to refer to spices of the India subcontinent, comprising the most popular notes in its cuisine. Often curry centered, with accents of cinnamon, clove and cumin."
	},
	{
		"id" : 825,
		"name" : "Licorice",
		"content" : "A sweet soft spicy anisic fragrance."
	},
	{
		"id" : 828,
		"name" : "Nutmeg",
		"content" : "It is a fruit of Myristica fragrans, it has a spicy sweet facet with earthy and pungent base notes, and is used to provide an oriental spiciness that is subtler than the usual cinnamon-clove-vanilla bouquet of orientals, thus perfect for masculine and lighter woody fragrances. Notable example is Fahrenheit by Dior and Vetiver Sport by Guerlain."
	},
	{
		"id" : 829,
		"name" : "Oily Notes",
		"content" : "Different materials give an oily effect, for example, narcissus."
	},
	{
		"id" : 831,
		"name" : "Pepper",
		"content" : "A hot and bracing note, short-lived and earthy spice, rendered through steam distillation of the berries of the piper negrum bush, which is very popular as a bright accent top note in fragrances. Featured prominently in Piper Negrum by L.Villoresi."
	},
	{
		"id" : 836,
		"name" : "Pink Pepper",
		"content" : "A fresh spicy floral and herbal top note with rosy nuance."
	},
	{
		"id" : 837,
		"name" : "Priprioca",
		"content" : "An earthy aromatic woody note."
	},
	{
		"id" : 838,
		"name" : "Saffron",
		"content" : "A refined note coming from stamens of Crocus sativus, a small flower in the Iris family known since antiquity. Its odor profile is bittersweet, leathery, soft and intimate, with an earthy base note."
	},
	{
		"id" : 839,
		"name" : "Safraleine",
		"content" : "the essence of saffron, leathery-suede soft with bittersweet tonalities"
	},
	{
		"id" : 840,
		"name" : "Sesame",
		"content" : "Fragrant note that recalls the sesame seeds of Sesamum indicum, familiar from its use in cuisine all over the world. Comforting, earthy, bread-like and soft."
	},
	{
		"id" : 842,
		"name" : "Sichuan Pepper",
		"content" : "a peppery note coming from the berries of the Zanthoxylum plant. It is aromatic, herbal, spicy, with a citrus touch."
	},
	{
		"id" : 844,
		"name" : "Spiked Pepper",
		"content" : "With a fougère green aroma, it exudes spiciness and oily nuances when crumbled."
	},
	{
		"id" : 845,
		"name" : "Star Anise",
		"content" : "A soft powdery floral spicy fragrance with nuances of licorice."
	},
	{
		"id" : 848,
		"name" : "Timur",
		"content" : "Spicy fresh, airy, slightly salty, reminiscent of grapefruit."
	},
	{
		"id" : 849,
		"name" : "Tonka Bean",
		"content" : "Tonka beans - fruits of Dipteryx odorata, a tropical tree native to Central and South America - contain 1-3% of coumarin. Tonka bean absolute contains 90% of coumarin. Depends on the amount, the scent varies from green grassy to sweet tobacco almond."
	},
	{
		"id" : 850,
		"name" : "Toscanol",
		"content" : "anisic molecule aimed to give an agrestic and sweet-spicy note to fragrances with hints of licorice, aiding diffusion and projection. It can substitute basil and tarragon"
	},
	{
		"id" : 852,
		"name" : "Vanilla",
		"content" : "An ever popular fragrance note, known mostly through its synthetic variant vanillin, which is sweet, cozy, comforting, with a pleasing cookie-baking feeling to it. Alongside amber, the reference note for the Oriental family of scents (The most famous classic being Shalimar). The real vanilla pod has darker facets that recall treacle and booze with off notes. Simple vanillas (Victoria's Secret Love to Dream, Charlie Touch, TBS Vanilla, Coty Vanilla Musk) have become increasingly popular with the adolescent market, giving rise to the umbiquity of the gourmand category of scents, while complex, earthier vanillas are appearing steadily in the niche sector (Spirituese Double Vanille by Guerlain, Tihota Indult, Montale Vanille Absolue)."
	},
	{
		"id" : 862,
		"name" : "Baklava",
		"content" : "An extremely sweet syrupy dessert with nuts, a fantasy note."
	},
	{
		"id" : 866,
		"name" : "Brioche",
		"content" : "Freshly baked sweet bread."
	},
	{
		"id" : 872,
		"name" : "Butter",
		"content" : "a creamy buttery note from different sources."
	},
	{
		"id" : 881,
		"name" : "Candies",
		"content" : "a sweet fantasy note that recalls sugar and hard candy with the scent of assorted fruits and flavors."
	},
	{
		"id" : 883,
		"name" : "Caramel",
		"content" : "A candy-deep, sensual and rich, creamy buttery lactonic note that enriches gourmand perfumes and adds sweetness to floral compositions."
	},
	{
		"id" : 898,
		"name" : "Cookie",
		"content" : "a sweet fantasy note."
	},
	{
		"id" : 902,
		"name" : "Cream",
		"content" : "A fantasy note or aspect of a number of ingredients."
	},
	{
		"id" : 905,
		"name" : "Croissant",
		"content" : "a fantasy gourmand note that has buttery facets but also a starch and baked feel."
	},
	{
		"id" : 906,
		"name" : "Cupcake",
		"content" : "a modern gourmand note that exploits the craziness for brightly colored desserts with cream and chocolate"
	},
	{
		"id" : 907,
		"name" : "Custard",
		"content" : "a sweet delicious cream."
	},
	{
		"id" : 908,
		"name" : "Dark Chocolate",
		"content" : "A deep and rich gourmand note, rather bitter than sweet."
	},
	{
		"id" : 909,
		"name" : "Dates",
		"content" : "a sweet fruity sugary note."
	},
	{
		"id" : 914,
		"name" : "French Pastries",
		"content" : "A sweet fantasy note of sweets with cream and fruits."
	},
	{
		"id" : 925,
		"name" : "Honeycomb",
		"content" : "A sweet floral scent of honey and beeswax."
	},
	{
		"id" : 926,
		"name" : "Horchata",
		"content" : "Nutty, sweet, little bit earthy (tiger nuts)."
	},
	{
		"id" : 927,
		"name" : "Ice cream",
		"content" : "a fantasy note of the sweet dessert."
	},
	{
		"id" : 935,
		"name" : "Macarons",
		"content" : "popular fantasy note recreating the bittersweet and savory feel of a macaron dessert, built on almond aroma and heliotrope essence."
	},
	{
		"id" : 937,
		"name" : "Maple Syrup",
		"content" : "a sweet and spicy note that is built on the fenugreek spice, helichrysum and on pyrazines, giving a toasted caramelic and savory note all at once"
	},
	{
		"id" : 940,
		"name" : "Marron Glacé",
		"content" : "A sweet and savory note of glazed chestnuts."
	},
	{
		"id" : 941,
		"name" : "Marshmallow",
		"content" : "a sweet sugary powdery note."
	},
	{
		"id" : 942,
		"name" : "Marzipan",
		"content" : "an almondy note with a bittersweet facet, reminiscent of the similarly-named almond paste used in decorating and filling desserts"
	},
	{
		"id" : 948,
		"name" : "Nougat",
		"content" : "nutty sweet dessert note that plays on hazelnut and almond tones"
	},
	{
		"id" : 954,
		"name" : "Panna Cotta",
		"content" : "milky and coffee note that recalls the famous Italian dessert"
	},
	{
		"id" : 959,
		"name" : "Popcorn",
		"content" : "a savory note, comfortable, smoky, and satisfying."
	},
	{
		"id" : 960,
		"name" : "Praline",
		"content" : "A modern synthetic accord with a soft, but intense sweet lactonic balsamic fragrance."
	},
	{
		"id" : 976,
		"name" : "Strawberry Syrup",
		"content" : "A sweet fruity note."
	},
	{
		"id" : 977,
		"name" : "Sugar",
		"content" : "A very popular sugary sweet accord can be also fruity, chocolate or compliment any other note."
	},
	{
		"id" : 991,
		"name" : "Agarwood (Oud)",
		"content" : "A pathological secretion of the aquillaria tree, a rich, musty woody-nutty scent that is highly prized in the Middle East. In commercial perfumery, it's safe to say all \"oud\" is a recreated synthetic note."
	},
	{
		"id" : 992,
		"name" : "Akigalawood",
		"content" : "A synthetic molecule reminiscent of patchouli with a hint of pepper and fine agarwood."
	},
	{
		"id" : 998,
		"name" : "Amyris",
		"content" : "A name referring to either amyris elemifera or amyris balsamifera. It has a fresh piney balsamic fragrance with a citrus nuance."
	},
	{
		"id" : 1011,
		"name" : "Birch",
		"content" : "Birch wood itself has a sweet woody sappy green fragrance. Birch tar is associated with leather fragrances (so called Russian leather)."
	},
	{
		"id" : 1013,
		"name" : "Black Spruce",
		"content" : "A conifer tree that gives a dark-smelling, fir-like coniferous smell to mostly natural perfumes."
	},
	{
		"id" : 1014,
		"name" : "Blackwood",
		"content" : "Coming from the East Asian Dalbergia latifolia species. An uncommon, earthy, woody \"note\"."
	},
	{
		"id" : 1016,
		"name" : "Brazilian Rosewood",
		"content" : "A rich floral-woody note, slightly spicy."
	},
	{
		"id" : 1017,
		"name" : "Buddha Wood",
		"content" : "woody, balsamic, mossy, deep, smoky, with vetiver and sandalwood facetes"
	},
	{
		"id" : 1018,
		"name" : "Buxus",
		"content" : "A bush also known as boxwood. Not very common, but useful as a woody note."
	},
	{
		"id" : 1019,
		"name" : "Cabreuva",
		"content" : "a delicate sweet woody smell with a fruity nuance."
	},
	{
		"id" : 1025,
		"name" : "Cedar",
		"content" : "An soft woody note coming from either the Atlas Mountains (Morocco) or the Virginia (US) cedarwood. There are also many cedar-smelling synthetics in use."
	},
	{
		"id" : 1030,
		"name" : "Clearwood",
		"content" : "A molecule by Firmenich: soft woody transparent, patchouli without dirty noise, earthy hue."
	},
	{
		"id" : 1031,
		"name" : "Cocobolo",
		"content" : "Fantasy note inspired by the bright reddish-brown decorative Dalbergia retusa wood."
	},
	{
		"id" : 1037,
		"name" : "Cypriol Oil or Nagarmotha",
		"content" : "A powerful earthy aromatic woody fragrance."
	},
	{
		"id" : 1041,
		"name" : "Driftwood",
		"content" : "a dry woody note with salty aromatic nuances."
	},
	{
		"id" : 1044,
		"name" : "Ebony Tree",
		"content" : "Evocative of the black decorative wood, it is a \"note\" appearing in niche compositions and woody fragrances where a darker element is desired."
	},
	{
		"id" : 1046,
		"name" : "Eucalyptus",
		"content" : "a camphorous aromatic woody note with fruity nuances."
	},
	{
		"id" : 1047,
		"name" : "Fig tree",
		"content" : "A complex, green powdery and silky note that evokes the milky, coconut-tinged sap of the fruit, the dark, bitter and dusty green of the leaves, as well as the bark of the tree and the earth underneath. Very popular in niche perfumery. The most celebrated examples are Premier Figuier (L'Artisan) and Philosykos (Diptyque)."
	},
	{
		"id" : 1048,
		"name" : "Fir",
		"content" : "Fir absolute is a sweet balsamic, aromatic green spicy fragrance."
	},
	{
		"id" : 1052,
		"name" : "Hinoki Wood",
		"content" : "Famously highlighted in Hinoki by Comme des Garcons, it is a type of cypress that is prized in Japan. Evergreen smelling with lemony facets."
	},
	{
		"id" : 1062,
		"name" : "Larch",
		"content" : "Woody note evocative of the conifer tree growing in the colder countries of the Norther Hemisphere. Very uncommon."
	},
	{
		"id" : 1063,
		"name" : "Lichen",
		"content" : "the same as oakmoss."
	},
	{
		"id" : 1068,
		"name" : "Manuka",
		"content" : "Related to the tea tree, it comes from the Leptospermum scoparium species."
	},
	{
		"id" : 1069,
		"name" : "Maple",
		"content" : "a sweetish woody note, reminiscent of maple syrup."
	},
	{
		"id" : 1070,
		"name" : "Massoia",
		"content" : "A milky-smelling woody note, famously used in Santal Massoia Hermes."
	},
	{
		"id" : 1071,
		"name" : "Mesquite Wood",
		"content" : "A strong and pleasant woody scent."
	},
	{
		"id" : 1072,
		"name" : "Muhuhu",
		"content" : "Woody note coming from the Brachylaena hutchinsii species of tree. Extremely uncommon."
	},
	{
		"id" : 1075,
		"name" : "Neem",
		"content" : "Woody note coming from the Azadirachta indica species of tree. Extremely uncommon."
	},
	{
		"id" : 1076,
		"name" : "Nootka",
		"content" : "woody, aromatic with hints of grapefruit"
	},
	{
		"id" : 1077,
		"name" : "Oak",
		"content" : "An aromatic woody fragrance with eathy nuances."
	},
	{
		"id" : 1079,
		"name" : "Olive Tree",
		"content" : "a fantasy note of a savory woody green fragrance."
	},
	{
		"id" : 1080,
		"name" : "Palisander Rosewood",
		"content" : "Redwood or palisander (refers to a number of beautifully nuanced timber) has a soft sweet woody fragrance."
	},
	{
		"id" : 1081,
		"name" : "Palo Santo",
		"content" : "a woody note with a fresh citrus touch."
	},
	{
		"id" : 1084,
		"name" : "Paperbark",
		"content" : "Also known as Melaleuca, an indigenous to Australia evergreen tree with bottle-brush flowers, in the myrtle family with fresh smelling, aromatic foliage."
	},
	{
		"id" : 1085,
		"name" : "Papyrus",
		"content" : "a powerful dry aromatic woody scent with smoky and leathery nuances."
	},
	{
		"id" : 1089,
		"name" : "Pear Tree",
		"content" : "Fantasy note evoking the tangy-sweet fruity and woody notes of the pear tree."
	},
	{
		"id" : 1093,
		"name" : "Plum Tree",
		"content" : "A woodier, more sturdy note that takes the succulent scent of plum and adds to it."
	},
	{
		"id" : 1105,
		"name" : "Sequoia",
		"content" : "Inspired by the mighty sequoia trees, a woody note that is gaining increased popularity in fragrances."
	},
	{
		"id" : 1106,
		"name" : "Siam",
		"content" : "From the Siamese cassia tree which bears small yellow blossoms, also known as senna with a particular herbal, bitter aroma."
	},
	{
		"id" : 1108,
		"name" : "Spruce",
		"content" : "A green woody balsamic fragrance with a sweetish undertone."
	},
	{
		"id" : 1113,
		"name" : "Teak Wood",
		"content" : "a fantasy \"blond\" woody note."
	},
	{
		"id" : 1116,
		"name" : "Thuja",
		"content" : "Woody note reminiscent of the thuja tree, which belongs in the cypress family. Also known as red cedar or white cedar, though it is not related to cedar botanically."
	},
	{
		"id" : 1118,
		"name" : "Vetiver",
		"content" : "Vetivers from different parts of the world differ a lot. For example, Haitian vetiver is clean and ethereal, while the Javanese one is smoky and dusty. If we have to generalize the scent, it is earthy, woody, green."
	},
	{
		"id" : 1130,
		"name" : "Benzoin",
		"content" : "A warm balsamic note reminiscent of vanilla."
	},
	{
		"id" : 1131,
		"name" : "Birch Tar",
		"content" : "One of the natural materials used for recreating a smell of leather."
	},
	{
		"id" : 1132,
		"name" : "Bisabolene",
		"content" : "Bisabolene has a warm, sweet-spicy-balsamic odour very typical of opoponax and \"oriental\" fragrance types. It finds use in artificial oil of bergamot, myrrh and lemon, and is an excellent fixative for neroli bases."
	},
	{
		"id" : 1146,
		"name" : "Estoraque",
		"content" : "The Estoraque is an intense herb that exudes freshness and lightness. It is traditionally used in smell baths in the Amazon, by those looking for renewal. It is believed that this ritual attracts good fluids, due to its green and refreshing olfactory characteristic, awakening a sense of energy and vitality."
	},
	{
		"id" : 1147,
		"name" : "Gurjun Balsam",
		"content" : "Fresh woody and spicy."
	},
	{
		"id" : 1148,
		"name" : "Incense",
		"content" : "Olibanum or Frankincense is usually meant by incense in a fragrance pyramid. It could also refer to a smoky note."
	},
	{
		"id" : 1149,
		"name" : "Labdanum",
		"content" : "A deep, powerful, leathery and ambery note."
	},
	{
		"id" : 1150,
		"name" : "Mastic or Lentisque",
		"content" : "a fresh balsamic, turpentine-like scent."
	},
	{
		"id" : 1153,
		"name" : "Myrrh",
		"content" : "One of the oldest known perfume ingredients, it has a warm balsamic, sweet aromatic spicy fragrance."
	},
	{
		"id" : 1155,
		"name" : "Olibanum (Frankincense)",
		"content" : "A fresh balsamic, somewhat green, woody and spicy fragrance with a fruity top note."
	},
	{
		"id" : 1157,
		"name" : "Opoponax",
		"content" : "An intensely sweet balsamic warm spicy fragrance."
	},
	{
		"id" : 1158,
		"name" : "Peru Balsam",
		"content" : "An intense woody coniferous aromatic ambery fragrance."
	},
	{
		"id" : 1160,
		"name" : "Poplar (Populus) Buds",
		"content" : "a peculiar balsamic green and bitter-sweet scent."
	},
	{
		"id" : 1161,
		"name" : "Resins",
		"content" : "A general balsamic amber spicy fragrance, which refers to different sources."
	},
	{
		"id" : 1163,
		"name" : "Styrax",
		"content" : "A dry balsamic note with floral and woody nuances (depending on the method of extraction)."
	},
	{
		"id" : 1168,
		"name" : "Amber",
		"content" : "A sweet, resinous, cozy and warm, often rather powdery note recreated from a mix of balsams, usually labdanum, benzoin, vanilla, styrax and fir or a combination of some of these. A default oriental note."
	},
	{
		"id" : 1169,
		"name" : "Amber Xtreme",
		"content" : "Amber, Woody, Dry."
	},
	{
		"id" : 1170,
		"name" : "Ambergris",
		"content" : "A product of the intestines of sperm whales found floating on the ocean and blanched by sea and sun with a skin-like salty and warm effect. Synthetically recreated today."
	},
	{
		"id" : 1173,
		"name" : "Ambretone",
		"content" : "Amber-like synthetic to replicate amber, orientalized notes in perfumes"
	},
	{
		"id" : 1174,
		"name" : "Ambrette (Musk Mallow)",
		"content" : "Ambrette seeds oil has a sweet floral musky fragrance."
	},
	{
		"id" : 1176,
		"name" : "Ambrocenide (Symrise)",
		"content" : "an extremely powerful and very sharp dry woody scent with ambery, slightly animalic and smokey agarwood tones"
	},
	{
		"id" : 1178,
		"name" : "Ambrostar",
		"content" : "woody amber agarwood\r\nA powerful long-lasting amber, woody (agarwood), odor profile. Highly diffusive."
	},
	{
		"id" : 1179,
		"name" : "Ambroxan",
		"content" : "A modern synthetic compound with a dry amber musky smell."
	},
	{
		"id" : 1180,
		"name" : "Animal Notes",
		"content" : "Different ingredients, synthetic and natural, can produce an animalic dirty and deep sensual effect."
	},
	{
		"id" : 1181,
		"name" : "Bacon",
		"content" : "a synthesized note that recreates the savory, tasty scent of roasted bacon."
	},
	{
		"id" : 1184,
		"name" : "Carrot Seeds",
		"content" : "a musky, fluffy, cosmetic note."
	},
	{
		"id" : 1185,
		"name" : "Castoreum",
		"content" : "a pungent and strong animal-derived note from the beaver, today synthetically recreated for ethical reasons. Smells leathery and very intense."
	},
	{
		"id" : 1189,
		"name" : "Civet",
		"content" : "Naturally the byproduct of the anal glands of exotic civet cats (technically no cats), which smells very pungent and fecal but which gives an amazing radiance and warmth to florals. Now synthetically replicated with civetone for ethical reasons."
	},
	{
		"id" : 1196,
		"name" : "Goat Hair",
		"content" : "natural essence coming from the tinctured hair of a rutting billy goat, used by natural perfumers as an ethically harvested animalic note. Warm and musk-like"
	},
	{
		"id" : 1199,
		"name" : "Hyraceum",
		"content" : "Essence from the small rodent hyrax's dried up crystalline fecal matter, combining olfactory facets of musk, civet and castoreum. \t\r\n\r\nOdor : animal, warm, phenolic, fecal, urinal"
	},
	{
		"id" : 1200,
		"name" : "Kephalis",
		"content" : "Synthetic note with woody-ambery facets, resembling cedar, that blends well with florals and tobacco, very popular with modern fragrances"
	},
	{
		"id" : 1202,
		"name" : "Leather",
		"content" : "Synthetic or naturally derived note of pungent characteristics, reminiscent of cured hides and leather goods. Usually rendered by birch tar or by synth isoquinolines."
	},
	{
		"id" : 1203,
		"name" : "Meat",
		"content" : "novelty note synthetically reproduced replicating the savory scent of cooked meat"
	},
	{
		"id" : 1204,
		"name" : "Milk",
		"content" : "A warm, comfortable gourmand, but not too sweet note."
	},
	{
		"id" : 1205,
		"name" : "Musk",
		"content" : "There is a wide variety of sources for musk smelling materials, including synthetic musks and natural ones, mainly obtained from plants, since deer musk is forbidden to obtain and sell. The musk is a secretion inside an internal pouch on the abdomen of the male deer. In order to get it, you have to kill the deer."
	},
	{
		"id" : 1206,
		"name" : "Muskrat",
		"content" : "Musky, animalic."
	},
	{
		"id" : 1209,
		"name" : "Sea Shells",
		"content" : "A natural or synthetic savory note."
	},
	{
		"id" : 1212,
		"name" : "Starfish",
		"content" : "A marine note."
	},
	{
		"id" : 1213,
		"name" : "Suede",
		"content" : "A synthetic note which replicates the soft, pliable, sensuous feel of suede on the skin. Soft, powdery, musky."
	},
	{
		"id" : 1214,
		"name" : "Sylkolide",
		"content" : "A musk amber synthetic molecule."
	},
	{
		"id" : 1215,
		"name" : "Tolu Balsam",
		"content" : "a sweet amber balsamic fragrance."
	},
	{
		"id" : 1216,
		"name" : "Truffle",
		"content" : "a very special gourmand warm and inviting fragrance."
	},
	{
		"id" : 1220,
		"name" : "Amaretto",
		"content" : "Fantasy note reminiscent of bitter almond (the actual liqueur is made from apricot pits which have an almondy nuance), usual in gourmand perfumes."
	},
	{
		"id" : 1226,
		"name" : "Bellini",
		"content" : "A fantasy note which has a peachy, sparkling effect in perfume compositions. Pairs well with creamy and feminine florals."
	},
	{
		"id" : 1230,
		"name" : "Buttered Rum",
		"content" : "Buttery, intense and giant distilled drink odor."
	},
	{
		"id" : 1233,
		"name" : "Calvados Drink",
		"content" : "Calvados is the name of a distilled alcoholic drink, originating in Lower Normandy, France, made from apples, from which cider is extracted, which is then fermented and distilled."
	},
	{
		"id" : 1234,
		"name" : "Campari",
		"content" : "Bitters fantasy note, useful for sparkle and vibrancy"
	},
	{
		"id" : 1235,
		"name" : "Cappuccino",
		"content" : "A smooth, delicious note of coffee and milky lactonic facets, useful in gourmand compositions."
	},
	{
		"id" : 1236,
		"name" : "Champagne",
		"content" : "A sparkling, fizzy note (aldehydes, citrus, fruits, spice)."
	},
	{
		"id" : 1240,
		"name" : "Coca-Cola",
		"content" : "A combined effect of the following essences: cinnamon, lime, lemon, orange, coriander, vanilla and nutmeg, much like in the actual soft drink."
	},
	{
		"id" : 1242,
		"name" : "Cognac",
		"content" : "A spicy rich, fruity oaky note wth amber woody undertones."
	},
	{
		"id" : 1243,
		"name" : "Cognac",
		"content" : "a variation on Cognac from Eastern Europe, complex, deep, oaky and with delicious depth"
	},
	{
		"id" : 1245,
		"name" : "Cream Soda",
		"content" : "a vanilla-laced soft drink note which can be sometimes detected in gourmand compositions."
	},
	{
		"id" : 1252,
		"name" : "Génépi",
		"content" : "Aromatic, smooth herbal liqueur."
	},
	{
		"id" : 1253,
		"name" : "Gin",
		"content" : "a classic juniper berry note which appears in gin liquor, bracing, mind-clearing, exhilarating, useful in woody, fougere and chypre compositions to give a piquant start."
	},
	{
		"id" : 1258,
		"name" : "Jasmine Tea",
		"content" : "a fresh fantasy note"
	},
	{
		"id" : 1262,
		"name" : "Lemonade",
		"content" : "classic tart lemon and fruity sweetness note which is refreshing and almost feels thirst-quenching in fragrances. Contrary to lemon essential oil and lemon rind it has no floral or resinous components."
	},
	{
		"id" : 1263,
		"name" : "Limoncello",
		"content" : "intense lemon, bittersweet fantasy note that is perfect to give a gourmand or modern oriental a tart and succulent opening"
	},
	{
		"id" : 1269,
		"name" : "Martini",
		"content" : "dry, sophisticated and fun fantasy note reminiscent of the classic cocktail, useful for dry and refreshing fragrance compositions"
	},
	{
		"id" : 1270,
		"name" : "Masala Chai",
		"content" : "A variation of the sweet, spicy tea blend which has soothing, comforting facets to its odor profile. Fantasy note which is featured in Omnia by Bvlgari."
	},
	{
		"id" : 1273,
		"name" : "Mocha",
		"content" : "Mocha is a coffee-based drink made with espresso, steamed milk, chocolate and whipped cream."
	},
	{
		"id" : 1274,
		"name" : "Mojito",
		"content" : "a fresh fantasy note reminiscent of mojito (lime, mint)."
	},
	{
		"id" : 1279,
		"name" : "Pina Colada",
		"content" : "tropical, sweet note that recalls the famous cocktail, with nuances of creamy coconut and fruity pineapple. Frequent in \"gourmand\" and \"fresh\" compositions."
	},
	{
		"id" : 1287,
		"name" : "Rhum Agricole",
		"content" : "A fantasy note of sugarcane juice rum"
	},
	{
		"id" : 1292,
		"name" : "Soy Milk",
		"content" : "Savory milky note."
	},
	{
		"id" : 1296,
		"name" : "Tequila",
		"content" : "fantasy note reminiscent of the drink made from blue agave, a cactus. Salty and bittersweet, used in a variety of tropical or fresgh fragrance blends."
	},
	{
		"id" : 1302,
		"name" : "Vodka",
		"content" : "a mild alcoholic note of slight aromatic piquancy."
	},
	{
		"id" : 1303,
		"name" : "Whiskey",
		"content" : "a complex note that comprises facets of sweet hay, oak barrels, chocolate, dried fruits, honey, and mild spices. Sophisticated, warm and deep, usual in orientals, woody fragrances and some chypres."
	},
	{
		"id" : 1304,
		"name" : "White Wine",
		"content" : "a fruity note reminiscent of white wine."
	},
	{
		"id" : 1306,
		"name" : "Wine must",
		"content" : "A fermented grape smell"
	},
	{
		"id" : 1307,
		"name" : "Pepperwood™",
		"content" : "Odor spicy, peppery, freesia, rose, balsamic."
	},
	{
		"id" : 1308,
		"name" : "Accord Eudora®",
		"content" : "Created by perfumer Joachim Correll, it smells like clean, very hydrated skin."
	},
	{
		"id" : 1311,
		"name" : "Aldehydes",
		"content" : "An abstract soapy, fatty, fresh smell, reminiscent of fresh laundry and frost."
	},
	{
		"id" : 1312,
		"name" : "Aluminum",
		"content" : "An artificial metallic fresh note."
	},
	{
		"id" : 1315,
		"name" : "Ambrofix™",
		"content" : "Ambrofix™ smell is clearly ambery. It's close to a mixture of Iso e Super and Ambrox DL. Woody and fresh, with a piney and nutty nuance. cedarwood, floral, leather-like, slightly tobacco note, and dry musk-like aftertaste. balsamic amber-like."
	},
	{
		"id" : 1318,
		"name" : "Amyl Salicylate",
		"content" : "a fresh fruity floral green fragrance reminiscent of unripe banana"
	},
	{
		"id" : 1321,
		"name" : "Aquozone",
		"content" : "Created by Firmenich, an aquatic marine scent."
	},
	{
		"id" : 1327,
		"name" : "Blood",
		"content" : "A disturbing fresh metallic nuance."
	},
	{
		"id" : 1328,
		"name" : "Boisiris",
		"content" : "Boisiris has an elegant powdery woody scent with orris and warm tobacco/amber understones."
	},
	{
		"id" : 1331,
		"name" : "Brown Scotch Tape",
		"content" : "Fantasy note with a glue and plastic scent used in avant-garde releases, such as Comme des Garcons"
	},
	{
		"id" : 1333,
		"name" : "Calone",
		"content" : "a synthetic note composed to reproduce the light, airy and aqueous feel of watermelon/melon."
	},
	{
		"id" : 1335,
		"name" : "Camphor",
		"content" : "white crystals that smell like mothballs, this note is used to render a bracing, intensely green and pungent note that cuts through the intense sweetness and decay of some white flowers. Especially infamous in Tubereuse Criminelle by Lutens and Carnal Flower by F.Malle where it balances the tuberose."
	},
	{
		"id" : 1337,
		"name" : "Canvas",
		"content" : "Earthy, moist, soggy, slightly bitter."
	},
	{
		"id" : 1338,
		"name" : "Caoutchouc",
		"content" : "It is a naturally sourced rubber material."
	},
	{
		"id" : 1339,
		"name" : "Cascalone",
		"content" : "A note of fresh water with a fruity hue."
	},
	{
		"id" : 1341,
		"name" : "Cashmeran",
		"content" : "A popular synthetic accord with a soft diffusive warm woody musky amber scent, reminiscent of cashmere wool."
	},
	{
		"id" : 1347,
		"name" : "Clay",
		"content" : "A note is reminiscent of terra-cotta tiles and the muddy scent of earth. Often a product of composition rather than a specific synthetic."
	},
	{
		"id" : 1355,
		"name" : "Coranol",
		"content" : "It is a Firmenich molecule with a fresh floral rosy scent with a slight metalic hue."
	},
	{
		"id" : 1357,
		"name" : "Cosmone",
		"content" : "A \"white musk\" scent with pleasantly sweetish, anisic undertones."
	},
	{
		"id" : 1358,
		"name" : "Coumarin",
		"content" : "the odor profile depends on the quantity of the material: it can be herbal like hay, or sweet and warm like caramelized almonds."
	},
	{
		"id" : 1361,
		"name" : "Cuban Cigar",
		"content" : "Tobacco burning, with a greenish and tenacious touch."
	},
	{
		"id" : 1364,
		"name" : "Dew Drop",
		"content" : "A fresh natural note of water usually paired with flowers or green notes."
	},
	{
		"id" : 1369,
		"name" : "Dust",
		"content" : "Dry, dusty, dense, closed and instigating odor."
	},
	{
		"id" : 1370,
		"name" : "Earth Tincture",
		"content" : "Essence used by perfumers to render a note of dirt, as in gardening or earth dust."
	},
	{
		"id" : 1374,
		"name" : "Ethyl Maltol",
		"content" : "a warm-fruity caramel sweet jam-like odor with an aspect of cotton candy."
	},
	{
		"id" : 1377,
		"name" : "Fabric",
		"content" : "A fantasy clean note."
	},
	{
		"id" : 1385,
		"name" : "Galaxolide",
		"content" : "Galaxolide is a synthetic musk with a lightweight clean pleasant somewhat fruity fragrance."
	},
	{
		"id" : 1388,
		"name" : "Geosmin",
		"content" : "Synthetic note, originally composed in nature by Streptomyces bacteria among other components such as the Germacradienol synthase enzyme.  It contributes the effect of rained upon earth, very easily detectable and beloved by humans."
	},
	{
		"id" : 1392,
		"name" : "Gunpowder",
		"content" : "Fantasy note with peppery, sulfurous and pyrocaustic/firecracker facets."
	},
	{
		"id" : 1395,
		"name" : "Head Space Waterfall",
		"content" : "A note of fresh water obtained by a head space technique."
	},
	{
		"id" : 1396,
		"name" : "Healingwood",
		"content" : "Healingwood has the deep, woody aspect of patchouli, with a much more radiant and vibrant aspect. It’s particularly useful in modern chypre notes, as it imparts this bright aspect which cannot be found in patchouli crude oil. It’s also highly impactful and complex."
	},
	{
		"id" : 1397,
		"name" : "Hedione",
		"content" : "A fresh floral fragrance reminiscent of jasmine with green nuances."
	},
	{
		"id" : 1402,
		"name" : "Hindinol",
		"content" : "Sandalwood, woody, milky, creamy, natural."
	},
	{
		"id" : 1407,
		"name" : "Indole",
		"content" : "in pure form indole smells like mothballs, camphor green and intense; in white flowers such as jasmine, tuberose and orange blossom it adds a deep animalic, slightly fecal tonality"
	},
	{
		"id" : 1409,
		"name" : "Ink",
		"content" : "A fantasy note that is reminiscent of India ink. Ink notes are also surfacing from natural materials such as oakmoss."
	},
	{
		"id" : 1413,
		"name" : "Iso E Super",
		"content" : "a synthetic woody cozy ambery musky note."
	},
	{
		"id" : 1417,
		"name" : "Javanol",
		"content" : "A synthetic sandalwood fragrance."
	},
	{
		"id" : 1422,
		"name" : "Lava",
		"content" : "A fantasy note of hot fiery lava."
	},
	{
		"id" : 1428,
		"name" : "Lorenox",
		"content" : "a complex accord developed by Mane which can be described as woody, ambery, leathery, aromatic."
	},
	{
		"id" : 1429,
		"name" : "Magnolan",
		"content" : "A Symrise molecule with a scent of natural magnolia (hints of citrus, lily of the valley, floral dewy freshness)."
	},
	{
		"id" : 1433,
		"name" : "Metallic Notes",
		"content" : "An artificial fresh fragrance."
	},
	{
		"id" : 1442,
		"name" : "Mugane",
		"content" : "A captive floral molecule by MANE."
	},
	{
		"id" : 1454,
		"name" : "Osmasylk Natsublim™",
		"content" : "Spicy aroma with a touch of clean skin, developed exclusively by AVON."
	},
	{
		"id" : 1457,
		"name" : "Para-Cresyl Phenyl Acetate",
		"content" : "Para-Cresyl Phenyl Acetate is a resemblance to sweet narcotic floral bouquet consisted of rose, narcissus, ylang-ylang, lily, hyacinth, narcissus, tuberose, lily-of-the-valley and so on. It also has  a urine, medicinal, phenolic and honey like scent."
	},
	{
		"id" : 1475,
		"name" : "Pollen",
		"content" : "A sweet powdery honey-like floral fragrance."
	},
	{
		"id" : 1476,
		"name" : "Pomarose",
		"content" : "A Givaudan captive material with a powerful rose fruity smell."
	},
	{
		"id" : 1477,
		"name" : "Porcelain",
		"content" : "Cold odor, slightly reminiscent of damp plaster."
	},
	{
		"id" : 1478,
		"name" : "Powdery Notes",
		"content" : "A vast variety of natural and synthetic notes give a powdery effect to perfume composition. Most usual powder notes are iris/orris, violet, vanilla, rose, musks, heliotrope, opoponax resin, and some amber materials."
	},
	{
		"id" : 1481,
		"name" : "Prunol",
		"content" : "Synthetic molecule that refers to the odor of a plum with tangy and finally oily aspects."
	},
	{
		"id" : 1486,
		"name" : "River Notes",
		"content" : "Natural smell of green touches, spicy with earthy nuances and reminiscent of fresh clay."
	},
	{
		"id" : 1489,
		"name" : "Salt",
		"content" : "A savory note that can render interesting a marine, woody or -more intriguingly- a gourmand composition."
	},
	{
		"id" : 1494,
		"name" : "Sclarene",
		"content" : "Sclarene brings a strong dry woody note with camphor and warm metallic undertones. It can be used in small quantity with musky and ambery notes such as Ambrofix to bring warm and metallic facets. Sclarene is also often considered as a musk booster as it shows interesting musky facets with good substantivity and fixative properties without being musky. Sclarene is commonly used at low dosages below 1% but can be overdosed to bring very unique signature to the fragrance."
	},
	{
		"id" : 1495,
		"name" : "Sea Water",
		"content" : "A fresh salty fragrance of the sea."
	},
	{
		"id" : 1496,
		"name" : "Serenolide",
		"content" : "Skin odor, clean musk."
	},
	{
		"id" : 1501,
		"name" : "Siren",
		"content" : "Saline, indole and gustatory odor."
	},
	{
		"id" : 1503,
		"name" : "Slate",
		"content" : "Fantasy note that is dry and modern."
	},
	{
		"id" : 1504,
		"name" : "Smoke",
		"content" : "Different ingredients can produce a smoky effect (birch tar, different incenses)."
	},
	{
		"id" : 1508,
		"name" : "Solar Notes",
		"content" : "An uplifting sparkling note of different sources."
	},
	{
		"id" : 1510,
		"name" : "Spiranol",
		"content" : "Firmenich molecule Spiranol has camphoreous, woody, green, cooling minty, earthy and tropical scent."
	},
	{
		"id" : 1515,
		"name" : "Suederal®",
		"content" : "Leather, Suede, Strong."
	},
	{
		"id" : 1528,
		"name" : "Tonalide®",
		"content" : "Musky. Berries. Earthy.\r\nIt is one of the most expensive musk."
	},
	{
		"id" : 1537,
		"name" : "Verdox",
		"content" : "Created in 1991 by IFF, Verdox ™ is a synthetic raw material which is often used to deliver an apple note to perfumes. The molecule also develops an intense woody facet. Verdox ™ is used in woody, coniferous, fruity-apple and fruity tea notes."
	},
	{
		"id" : 1539,
		"name" : "Vinyl Guaiacol",
		"content" : "Provide the smell of vanilla-cloves accord, sweet balsamic and smokey, spicy, powdery carnation accord."
	}
]


class FragranceRecommender:
    def __init__(self):
        # 향 계열과 향료 데이터 초기화
        self.fragrance_lines = FRAGRANCE_LINES
        self.fragrance_notes = FRAGRANCE_NOTES
        
        # 임베딩 모델 초기화
        self.model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')
        
        # GPT 모델 초기화
        self.chat_model = ChatOpenAI(model="gpt-4o", temperature=0.7)
        
        # 프롬프트 템플릿 수정
        self.prompt_template = """사용자가 원하는 느낌: {user_feeling}

[추천 향 계열과 어울리는 향료]
{recommendations}

위 향 계열들과 그에 어울리는 향료들이 사용자의 느낌과 어떻게 매칭되는지 자연스럽게 설명해주세요.
각 향 계열별로 어떤 향료 조합이 잘 어울리는지, 이 조합이 어떤 분위기를 만들어내는지 설명해주세요.
"""
        self.prompt = PromptTemplate.from_template(self.prompt_template)
        
        # 임베딩 초기화
        self.calculate_embeddings()

    def _preprocess_text(self, text):
        """텍스트 전처리"""
        text = re.sub(r'[^\w\s\-가-힣]', ' ', text)
        text = ' '.join(text.split())
        return text.lower()
    
    def calculate_embeddings(self):
        """모든 임베딩 계산"""
        # 향 계열 임베딩
        valid_lines = []
        line_texts = []
        
        for line in self.fragrance_lines:
            if line["name"] != "notname":
                text = f"{line['name']} {line['content']}"
                line_texts.append(self._preprocess_text(text))
                valid_lines.append(line)
        
        self.line_embeddings = self.model.encode(line_texts)
        self.valid_lines = valid_lines
        
        # 향료 임베딩
        valid_notes = []
        note_texts = []
        
        for note in self.fragrance_notes:
            text = f"{note['name']} {note['content']}"
            note_texts.append(self._preprocess_text(text))
            valid_notes.append(note)
        
        self.note_embeddings = self.model.encode(note_texts)
        self.valid_notes = valid_notes

    def _find_matching_notes_for_line(self, line, top_k=3, threshold=0.2):
        """특정 향 계열에 어울리는 향료 찾기"""
        line_text = f"{line['name']} {line['content']}"
        line_embedding = self.model.encode(self._preprocess_text(line_text))
        
        # 향료와의 유사도 계산
        similarities = cosine_similarity([line_embedding], self.note_embeddings)[0]
        note_indices = np.argsort(similarities)[-top_k:][::-1]
        
        matching_notes = [
            {
                **self.valid_notes[i],
                'similarity': float(similarities[i])
            }
            for i in note_indices
            if similarities[i] > threshold
        ]
        
        return matching_notes

    def get_recommendations(self, user_feeling, top_k_lines=3, threshold=0.2):
        """사용자 입력에 기반한 향 계열과 그에 어울리는 향료 추천"""
        query = self._preprocess_text(user_feeling)
        query_embedding = self.model.encode(query)
        
        # 향 계열 유사도 계산 및 상위 3개 선택
        line_similarities = cosine_similarity([query_embedding], self.line_embeddings)[0]
        line_indices = np.argsort(line_similarities)[-top_k_lines:][::-1]
        
        recommendations = []
        for i in line_indices:
            if line_similarities[i] > threshold:
                line = self.valid_lines[i]
                line_with_similarity = {
                    **line,
                    'similarity': float(line_similarities[i])
                }
                # 해당 향 계열에 어울리는 향료 찾기
                matching_notes = self._find_matching_notes_for_line(line)
                recommendations.append({
                    'line': line_with_similarity,
                    'notes': matching_notes
                })
        
        if not recommendations:
            return {
                "success": False,
                "message": "입력하신 느낌에 맞는 향 계열을 찾을 수 없습니다. 다른 느낌을 입력해 주세요."
            }
        
        # GPT 추천 설명 생성
        recommendation_text = self._generate_gpt_recommendation(user_feeling, recommendations)
        
        return {
            "success": True,
            "recommendations": recommendations,
            "recommendation_text": recommendation_text
        }
    
    def _generate_gpt_recommendation(self, user_feeling, recommendations):
        """GPT를 사용한 추천 설명 생성"""
        recommendations_text = []
        
        for rec in recommendations:
            line = rec['line']
            notes = rec['notes']
            
            line_text = f"\n[{line['name']} (유사도: {line['similarity']:.1%})]\n"
            line_text += f"계열 설명: {line['content']}\n"
            
            if notes:
                line_text += "어울리는 향료:\n"
                for note in notes:
                    line_text += f"- {note['name']}: {note['content']} (유사도: {note['similarity']:.1%})\n"
            else:
                line_text += "매칭되는 향료가 없습니다.\n"
                
            recommendations_text.append(line_text)
        
        # GPT에 전달할 프롬프트 생성 및 응답 받기
        recommendation = self.chat_model.predict(
            self.prompt.format(
                user_feeling=user_feeling,
                recommendations="\n".join(recommendations_text)
            )
        )
        
        return recommendation

def main():
    recommender = FragranceRecommender()
    
    while True:
        user_feeling = input("\n원하는 향의 느낌을 입력해주세요 (종료하려면 'q' 입력): ")
        
        if user_feeling.lower() == 'q':
            print("\n프로그램을 종료합니다.")
            break
        
        result = recommender.get_recommendations(user_feeling)
        
        print("\n" + "="*50)
        if result["success"]:
            # GPT 추천 설명 출력
            print("[AI 추천 분석]")
            print(result["recommendation_text"])
            
            # 상세 정보 출력
            print("\n[상세 매칭 정보]")
            for rec in result["recommendations"]:
                line = rec['line']
                print(f"\n• {line['name']} 계열")
                print(f"  설명: {line['content']}")
                print(f"  색상: #{line['color']}")
                print(f"  유사도: {line['similarity']:.1%}")
                
                if rec['notes']:
                    print("\n  어울리는 향료:")
                    for note in rec['notes']:
                        print(f"    - {note['name']}")
                        print(f"      설명: {note['content']}")
                        print(f"      유사도: {note['similarity']:.1%}")
                print("-" * 30)
        else:
            print(result["message"])
        print("="*50)

if __name__ == "__main__":
    main()


[AI 추천 분석]
결혼식에 어울리는 향을 찾는다면, 몇 가지 향 계열과 그에 어울리는 향료 조합을 고려해 볼 수 있습니다. 각각의 계열은 특별한 분위기를 만들어내며, 결혼식의 성격에 맞게 선택할 수 있습니다.

### 1. Floral 계열
Floral 계열은 화려하고 여성스러운 느낌을 주는 향으로, 결혼식의 로맨틱한 분위기와 잘 어울립니다. 이 계열에서 추천하는 향료 조합은 Plum, Carissa, 그리고 Lily입니다.

- **Plum**: 부드럽고 감각적인 과일향으로, Floral 계열의 풍부한 꽃 향기를 보완하며 따뜻하고 달콤한 느낌을 더합니다.
- **Carissa**: 자스민과 가드니아를 연상시키는 달콤한 흰 꽃 향기로, 과일향과 어우러져 풍부하고 우아한 향을 만들어냅니다.
- **Lily**: 매콤하고 왁시한 꽃 향기로, 전체적인 향 조합에 깊이와 고급스러움을 더해줍니다.

이 조합은 신부의 우아함과 결혼식의 로맨틱한 분위기를 한층 더 돋보이게 합니다.

### 2. Gourmand 계열
Gourmand 계열은 달콤하고 유혹적인 향으로, 결혼식의 따뜻하고 포근한 순간에 잘 어울립니다. 이 계열에서 추천하는 향료 조합은 Plum, Truffle, 그리고 Caramel입니다.

- **Plum**: 감각적인 과일향으로, Gourmand 계열의 달콤함을 더욱 풍부하게 만들어 줍니다.
- **Truffle**: 따뜻하고 초대하는 느낌의 특별한 고메 향으로, 결혼식의 특별한 순간을 더욱 기억에 남게 합니다.
- **Caramel**: 부드럽고 감미로운 캐러멜 향이 Floral 향과 결합되어 풍성하고 달콤한 분위기를 연출합니다.

이 조합은 달콤하고 유혹적인 분위기를 만들어, 결혼식의 따뜻한 감성을 극대화시킵니다.

### 3. Chypre 계열
Chypre 계열은 격조 있고 성숙한 느낌을 주는 향으로, 성숙한 결혼식 분위기와 잘 어울립니다. 이 계열에서 추천하는 향료 조합은 Cabreuva, Plum, 그리고 Cosmone입니다.

- **Cabre

In [ ]:
!pip install sentence-transformers scikit-learn numpy langchain openai

In [35]:
import requests
from PIL import Image
from io import BytesIO
from transformers import BlipProcessor, BlipForConditionalGeneration
import mysql.connector
from langchain_openai import ChatOpenAI

# MySQL 데이터베이스 연결을 위한 함수
def get_mysql_connection():
    try:
        connection = mysql.connector.connect(
            host="localhost",
            user="ohgiraffers",
            password="ohgiraffers",
            database="test_db"
        )
        return connection
    except mysql.connector.Error as err:
        print(f"DB 연결 오류: {err}")
        return None

# 이미지를 다운로드하고 BLIP 모델을 사용해 캡션을 생성하는 함수
def blip_image_caption(image_url):
    try:
        response = requests.get(image_url)
        if response.status_code == 200:
            image = Image.open(BytesIO(response.content))
        else:
            print(f"이미지 다운로드 실패, 상태 코드: {response.status_code}")
            return "이미지를 다운로드할 수 없습니다."
        
        # BLIP 모델 로드
        processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
        model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

        # 이미지 캡션 생성
        inputs = processor(image, return_tensors="pt")
        out = model.generate(**inputs, max_new_tokens=500)
        caption = processor.decode(out[0], skip_special_tokens=True)
        
        print("이미지 캡션:", caption)
        return caption

    except Exception as e:
        print(f"BLIP 모델 오류: {e}")
        return "이미지를 분석할 수 없습니다."

# 향수 데이터베이스에서 향수 추천을 가져오는 함수
def fetch_perfume_recommendations(caption):
    connection = get_mysql_connection()
    if not connection:
        return []
    cursor = connection.cursor(dictionary=True)
    
    # 최대 5개의 키워드를 추출하여 향수 검색
    keywords = caption.split()[:5]
    like_conditions = " OR ".join(["description LIKE %s"] * len(keywords))
    query = f"""
    SELECT name, brand, description
    FROM perfume
    WHERE {like_conditions}
    LIMIT 3;
    """
    cursor.execute(query, tuple([f"%{keyword}%" for keyword in keywords]))
    results = cursor.fetchall()
    cursor.close()
    connection.close()

    # 향수 결과 출력
    if results:
        print(f"데이터베이스에서 찾은 향수들: {results}")
    else:
        print("향수 결과 없음")

    return results

# GPT를 사용하여 향수 특징과 느낌 생성
def generate_perfume_characteristics_and_feeling_with_gpt(perfumes, user_feeling):
    try:
        llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)
        
        perfume_info = "\n".join([f"{perfume['name']} - {perfume['brand']}: {perfume['description']}" for perfume in perfumes])
        
        prompt_template = f"""
        사용자가 '{user_feeling}'라는 느낌을 요청했습니다.
        아래는 데이터베이스에서 찾은 향수 목록입니다:
        {perfume_info}
        
        위의 목록에서 사용자의 느낌에 맞는 향수를 추천하고, 선택한 이유를 설명해주세요.
        절대로 목록에 없는 다른 향수를 추천하지 마세요.
        
        응답 형식:
        추천 향수:
        1. [향수 이름] - [브랜드]
        2. [향수 이름] - [브랜드]
        3. [향수 이름] - [브랜드]
        
        추천 이유:
        각각의 향수가 사용자 느낌과 부합하는 이유를 간단히 설명해주세요.
        """
        
        response = llm(messages=[ 
            {"role": "system", "content": "당신은 1등급 조향사입니다. 주어진 향수 목록을 분석하고, 사용자가 요청한 느낌에 맞는 향수를 추천하고 이유를 설명해주세요."},
            {"role": "user", "content": prompt_template}
        ])
        
        print("GPT 응답:", response)  # 응답 전체 로그 출력
        
        if response and 'choices' in response and len(response['choices']) > 0:
            content = response['choices'][0].get('message', {}).get('content', "응답 형식이 잘못되었습니다.")
            return content
        else:
            return "GPT 응답 오류: 잘못된 응답 형식"
    
    except Exception as e:
        print(f"GPT 모델 오류: {e}")
        return "향수 분석 오류"
    

# 응답 내용을 분리하는 함수
def recommend_fragrance_based_on_image(image_url):
    caption = blip_image_caption(image_url)
    perfumes = fetch_perfume_recommendations(caption)
    
    if not perfumes:
        return "이미지에서 추출한 느낌에 맞는 향수를 찾을 수 없습니다."
    
    # 결과 출력
    print(f"\n[이미지 설명]: {caption}\n")


# 사용자로부터 이미지 URL을 입력받고 추천 결과 출력
image_url = input("이미지 URL을 입력하세요: ")
recommend_fragrance_based_on_image(image_url)

preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

c:\Users\20118\miniforge3\envs\langchain_env\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\20118\.cache\huggingface\hub\models--Salesforce--blip-image-captioning-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

이미지 캡션: a yellow apple with leaves on a white background
데이터베이스에서 찾은 향수들: [{'name': '퓌메리 튀르크 오 드 퍼퓸', 'brand': '세르주 루텐', 'description': 'Serge Lutens의 Gratte-ciel 컬렉션에 포함된 블론드, 허니 및 스모크 향'}, {'name': '페리유스몬 보터 오 드 퍼퓸', 'brand': '세르주 루텐', 'description': 'Gratte-ciel 컬렉션에 포함된 결단력 있고 용기 있고 화려한 향수'}, {'name': '뀌르 모레스크 오 드 퍼퓸', 'brand': '세르주 루텐', 'description': 'Serge Lutens의 Gratte-ciel 컬렉션에 포함된 세련되고 동물적이며 관능적인 향수입니다.'}]

[이미지 설명]: a yellow apple with leaves on a white background



In [ ]:
import json
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

# JSON 파일 로드 함수
def load_json(filename):
    with open(filename, 'r', encoding='utf-8') as f:
        return json.load(f)

# 데이터 로드
categories = load_json('line.json')
perfumes = load_json('perfume.json')
spices = load_json('spice 계열추가.json')

def infer_category_from_input_gpt(user_input):
    """사용자 입력을 바탕으로 향수 계열 추론"""
    prompt = PromptTemplate(
        input_variables=["user_input"],
        template="""
        사용자의 향수 선호도를 분석하여 가장 적합한 향수 계열의 ID를 선택해주세요.
        
        계열 ID 목록:
        1: Spicy (스파이시)
        2: Cypre (시프레)
        3: Fruity (프루티)
        4: Citrus (시트러스)
        5: Green (그린)
        6: Aldehyde (알데하이드)
        7: Aquatic (아쿠아틱)
        8: Fougere (푸제르)
        9: Gourmand (구르망)
        10: Woody (우디)
        11: Oriental (오리엔탈)
        12: Floral (플로럴)
        13: Musk (머스크)
        14: Powdery (파우더리)
        15: Tobacco Leather (타바코 레더)
        16: Amber (앰버)
        
        사용자 입력: {user_input}
        
        응답 형식: 숫자만 작성 (예: 4)
        """
    )

    model = ChatOpenAI(model="gpt-4", temperature=0.3)
    response = model.invoke(prompt.format(user_input=user_input))
    
    try:
        category_id = int(''.join(filter(str.isdigit, response.content)))
        return category_id
    except:
        return 4  # 기본값

def get_category_spices(category_id, spices):
    """특정 계열에 속하는 향료 목록 반환"""
    return [spice['name'].lower() for spice in spices if category_id in spice['lindId']]

def get_perfume_notes(perfume):
    """향수의 모든 노트를 추출"""
    notes = set()
    
    if perfume.get('single_note'):
        notes.update(note.strip().lower() for note in perfume['single_note'].split(','))
    else:
        for note_type in ['top_note', 'middle_note', 'base_note']:
            if perfume.get(note_type):
                notes.update(note.strip().lower() for note in perfume[note_type].split(','))
    
    return notes

def count_matching_notes(perfume_notes, category_spices):
    """향수 노트와 계열 향료 간의 매칭 수 계산"""
    count = 0
    matched_notes = set()
    
    for note in perfume_notes:
        for spice in category_spices:
            if spice in note or note in spice:
                count += 1
                matched_notes.add(spice)
    
    return count, matched_notes

def recommend_fragrances(user_input):
    # 1. 사용자 입력으로부터 계열 추론
    category_id = infer_category_from_input_gpt(user_input)
    category_name = next((cat['name'] for cat in categories if cat['id'] == category_id), "Unknown")
    
    # 2. 해당 계열의 향료 목록 가져오기
    category_spices = get_category_spices(category_id, spices)
    
    # 3. 각 향수별로 계열 향료 매칭 검사
    perfume_matches = []
    for perfume in perfumes:
        perfume_notes = get_perfume_notes(perfume)
        match_count, matched_notes = count_matching_notes(perfume_notes, category_spices)
        
        if match_count > 0:  # 매칭되는 향료가 있는 경우만 추가
            perfume_matches.append({
                'perfume': perfume,
                'match_count': match_count,
                'matched_notes': matched_notes
            })
    
    # 4. 매칭 수를 기준으로 정렬
    perfume_matches.sort(key=lambda x: x['match_count'], reverse=True)
    
    # 5. 결과 포맷팅
    result = f"\n[추천 계열] {category_name}\n"
    result += f"[추천 이유] {category_name} 계열의 향료가 가장 많이 포함된 향수들입니다.\n"
    result += "[추천 향수]"
    
    # 상위 3개 향수 추천
    for i, match in enumerate(perfume_matches[:3], 1):
        perfume = match['perfume']
        result += f"\n{i}. {perfume['brand']} - {perfume['name']}"
        result += f"\n   특징: {perfume['description']}"
        result += f"\n   포함된 {category_name} 계열 향료: {', '.join(match['matched_notes'])}"
        result += f"\n   매칭 향료 수: {match['match_count']}"
    
    return result

def main():
    user_input = input("어떤 향수를 찾으시나요? ")
    result = recommend_fragrances(user_input)
    print(result)

if __name__ == "__main__":
    main()

INFO:__main__:Processing request for feeling: 시원한 바닷가같은 향
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:AI selected category: 시트러스
INFO:__main__:Selection reason: 선택한 계열: 시트러스
선택 이유: 시트러스 계열의 향료는 베르가못, 레몬, 오렌지, 자몽, 라임, 만다린, 탠저린, 시트러스 등을 주요 성분으로 하며, 이들은 대체로 상큼하고 신선한 향을 가지고 있습니다. "시원한 바닷가"라는 표현은 신선하고 청량감 있는 느낌을 주는 향을 원하는 것으로 해석될 수 있으며, 이러한 특성은 시트러스 계열의 향료와 잘 맞습니다. 특히, 레몬이나 자몽 같은 성분은 바다의 소금기와도 잘 어울리며, 이러한 이유로 시트러스 계열을 선택하였습니다.
INFO:__main__:Selected category: 시트러스
INFO:__main__:Searching for perfumes with notes: ['Bergamot', 'Lemon', 'Orange', 'Grapefruit', 'Lime', 'Mandarin', 'Tangerine', 'Citrus']
INFO:__main__:Found 0 perfumes



[추천 결과]
----------------------------------------
현재 데이터베이스에서 해당 향료를 가진 향수를 찾지 못했습니다.


In [11]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [28]:
import json
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

def load_json(filename):
    with open(filename, 'r', encoding='utf-8') as f:
        return json.load(f)

def get_all_notes(perfume):
    """향수의 모든 노트를 추출"""
    notes = []
    
    # single_note 처리
    if perfume.get('single_note'):
        if perfume['single_note']:  # None이 아닌 경우에만 처리
            notes.extend([note.strip().lower() for note in perfume['single_note'].split(',')])
    
    # top, middle, base notes 처리
    for note_type in ['top_note', 'middle_note', 'base_note']:
        if perfume.get(note_type):  # 해당 노트 타입이 있는지 확인
            if perfume[note_type]:  # None이 아닌 경우에만 처리
                notes.extend([note.strip().lower() for note in perfume[note_type].split(',')])
    
    return list(set(notes))  # 중복 제거

def get_category_name(category_id, categories):
    """카테고리 ID로 이름 찾기"""
    for category in categories:
        if category['id'] == category_id:
            return category['name']
    return "Unknown"

def get_category_spices(category_id, spices):
    """특정 계열의 향료 목록 가져오기"""
    category_spices = []
    for spice in spices:
        if category_id in spice['lindId']:
            category_spices.append(spice['name'].lower())
    return category_spices

def count_matching_spices(perfume_notes, category_spices):
    """향수 노트와 계열 향료 간의 매칭 카운트"""
    matched_spices = set()
    for note in perfume_notes:
        for spice in category_spices:
            if spice in note or note in spice:
                matched_spices.add(spice)
    return len(matched_spices), matched_spices

def recommend_fragrances(user_input, perfumes, spices, categories):
    """향수 추천 메인 함수"""
    # 1. 사용자 입력으로 카테고리 추론
    prompt = PromptTemplate(
        input_variables=["user_input"],
        template="""사용자의 향수 선호도를 분석하여 가장 적합한 향수 계열의 ID를 선택해주세요.
        계열 ID 목록:
        1: Spicy (스파이시)
        2: Cypre (시프레)
        3: Fruity (프루티)
        4: Citrus (시트러스)
        5: Green (그린)
        6: Aldehyde (알데하이드)
        7: Aquatic (아쿠아틱)
        8: Fougere (푸제르)
        9: Gourmand (구르망)
        10: Woody (우디)
        11: Oriental (오리엔탈)
        12: Floral (플로럴)
        13: Musk (머스크)
        14: Powdery (파우더리)
        15: Tobacco Leather (타바코 레더)
        16: Amber (앰버)
        
        사용자 입력: {user_input}
        응답 형식: 숫자만 작성"""
    )
    
    model = ChatOpenAI(model="gpt-4", temperature=0.3)
    response = model.invoke(prompt.format(user_input=user_input))
    try:
        category_id = int(''.join(filter(str.isdigit, response.content)))
    except:
        category_id = 4  # 기본값
    
    # 2. 카테고리 이름과 향료 가져오기
    category_name = get_category_name(category_id, categories)
    category_spices = get_category_spices(category_id, spices)
    
    # 3. 각 향수별로 매칭되는 향료 카운트
    perfume_matches = []
    for perfume in perfumes:
        try:
            notes = get_all_notes(perfume)
            match_count, matched_spices = count_matching_spices(notes, category_spices)
            
            if match_count > 0:
                perfume_matches.append({
                    'perfume': perfume,
                    'match_count': match_count,
                    'matched_spices': matched_spices
                })
        except Exception as e:
            print(f"Error processing perfume {perfume.get('name', 'unknown')}: {str(e)}")
            continue
    
    # 4. 매칭 수로 정렬
    perfume_matches.sort(key=lambda x: x['match_count'], reverse=True)
    
    # 5. 결과 포맷팅
    if not perfume_matches:
        return f"\n[추천 계열] {category_name}\n해당 계열의 향수를 찾을 수 없습니다."
    
    result = f"\n[추천 계열] {category_name}\n"
    result += f"[추천 이유] {category_name} 계열의 향료가 가장 많이 포함된 향수들입니다.\n"
    result += "[추천 향수]"
    
    for i, match in enumerate(perfume_matches[:3], 1):
        perfume = match['perfume']
        result += f"\n{i}. {perfume['brand']} - {perfume['name']}"
        result += f"\n   특징: {perfume['description']}"
        result += f"\n   포함된 {category_name} 계열 향료: {', '.join(match['matched_spices'])}"
        result += f"\n   매칭 향료 수: {match['match_count']}"
    
    return result

def main():
    # 데이터 로드
    perfumes = load_json('perfume.json')
    spices = load_json('spice 계열추가.json')
    categories = load_json('line.json')
    
    # 사용자 입력
    user_input = input("어떤 향수를 찾으시나요? ")
    
    # 향수 추천
    result = recommend_fragrances(user_input, perfumes, spices, categories)
    print(result)

if __name__ == "__main__":
    main()


[추천 계열] Floral
[추천 이유] Floral 계열의 향료가 가장 많이 포함된 향수들입니다.
[추천 향수]
1. 크리드 - 그린 네롤리 오 드 퍼퓸
   특징: 날카로움과 달콤함이 완벽한 균형을 이루면서 상쾌하고 시원한 시트러스 향을 전달하는 향수
   포함된 Floral 계열 향료: lavender, gardenia, orange blossom, rose, peony, tiare flower, lily, magnolia, jasmine
   매칭 향료 수: 9
2. 불리1803 - 오 트리쁠 리켄 데코스
   특징: 불리의 시그니처 향으로 수분 가득한 이끼 숲을 거니는 듯한 착각을 주는 향
   포함된 Floral 계열 향료: lavender, gardenia, orange blossom, rose, peony, tiare flower, lily, magnolia, jasmine
   매칭 향료 수: 9
3. 톰 포드 - 아주르 라임 오 드 퍼퓸
   특징: 세련됨과 함께 진화하는 캐주얼 엘레강스 시트러스 시프레 향
   포함된 Floral 계열 향료: lavender, gardenia, orange blossom, rose, peony, tiare flower, lily, magnolia, jasmine
   매칭 향료 수: 9


In [2]:
import json
from typing import List

# 향수 매칭 함수
def recommend_fragrances(user_input: str, perfumes: List[dict], fragrance_data: dict):
    # 계열 데이터 추출
    try:
        categories = fragrance_data  # 여기서 'fragrance_data'는 전체 데이터를 의미함
    except KeyError:
        return "계열 데이터가 존재하지 않습니다. JSON 파일의 구조를 확인해주세요."
    
    # 계열 목록을 템플릿에 맞게 포맷팅
    categories_list = "\n".join(
        [f"{category['name']}: {category['description']} (주요 성분: {', '.join(category['key_ingredients'])})" for category in categories]
    )
    
    # 계열 목록 출력
    print(f"계열 목록: {categories_list}")  # 디버깅용

    # 사용자 입력에 대해 계열을 추천하는 모델 템플릿
    # (GPT 모델 사용 부분 삭제)

    # 계열 ID 추론 과정 (수동 입력 또는 다른 로직으로 대체)
    selected_category = categories[0]  # 임시로 첫 번째 계열을 선택
    
    category_name = selected_category['name']
    category_description = selected_category['description']
    category_notes = selected_category['key_ingredients']

    # 향수 매칭 과정
    perfume_matches = []
    for perfume in perfumes:
        notes = extract_notes(perfume)
        match_count, matched_notes = count_matching_spices(notes, category_notes)
        print(f"향수: {perfume['name']}, 매칭된 노트 개수: {match_count}, 매칭된 노트: {matched_notes}")  # 디버깅용
        if match_count > 0:
            perfume_matches.append({
                'perfume': perfume,
                'match_count': match_count,
                'matched_notes': matched_notes
            })

    # 매칭된 향수 리스트를 매칭된 노트 수를 기준으로 정렬
    perfume_matches.sort(key=lambda x: x['match_count'], reverse=True)

    # 매칭된 향수가 없으면 결과 반환
    if not perfume_matches:
        return f"\n[추천 계열] {category_name}\n해당 계열의 향수를 찾을 수 없습니다."

    # 공통된 노트를 추출하여 추천 이유 생성
    common_notes = set()
    for match in perfume_matches[:3]:
        common_notes.update(match['matched_notes'])
    common_notes_text = ", ".join(common_notes) if common_notes else "계열의 대표 향료들"

    reason = (
        f"{category_name} 계열의 향수는 '{category_description}'을 주제로 하며, "
        f"사용자가 입력한 '{user_input}'과 같은 느낌을 표현하기에 적합합니다. "
        f"주요 노트: {common_notes_text}."
    )

    # 결과 생성
    result = f"\n[추천 계열] {category_name}\n"
    result += f"[추천 느낌] {reason}\n"
    result += "[추천 향수]\n"
    for i, match in enumerate(perfume_matches[:3], 1):
        perfume = match['perfume']
        result += f"{i}. {perfume['name']} : {perfume['brand']} : {perfume['description']}\n"
        result += f"   매칭 향료: {', '.join(match['matched_notes'])} (총 {match['match_count']}개)\n"
    
    return result

# 향수 데이터와 계열 데이터를 파일에서 읽어오는 함수
def load_data():
    # refined_fragrance_database.json 읽기
    with open('refined_fragrance_database.json', 'r', encoding='utf-8') as f:
        fragrance_data = json.load(f)
    
    # perfume.json 읽기
    with open('perfume.json', 'r', encoding='utf-8') as f:
        perfumes = json.load(f)
    
    return perfumes, fragrance_data

# 향수 노트 추출 함수 (예시)
def extract_notes(perfume):
    # 각 노트가 None이 아닐 경우만 split을 호출하고, None이면 빈 리스트를 반환
    top_notes = perfume['top_note'].split(', ') if perfume['top_note'] else []
    middle_notes = perfume['middle_note'].split(', ') if perfume['middle_note'] else []
    base_notes = perfume['base_note'].split(', ') if perfume['base_note'] else []
    
    # 모든 노트를 합쳐서 반환
    return top_notes + middle_notes + base_notes

# 향수 매칭 함수 (간단한 예시)
def count_matching_spices(notes, category_notes):
    matched_notes = [note for note in notes if note in category_notes]
    return len(matched_notes), matched_notes

# 사용자 입력 예시
user_input = "꽃향기와 상큼한 느낌을 주는 향수 추천해주세요."

# 향수와 계열 데이터를 파일에서 읽어오기
perfumes, fragrance_data = load_data()

# 함수 호출 및 결과 출력
print(recommend_fragrances(user_input, perfumes, fragrance_data))

계열 목록: 자극적 (Spicy): 자극적이며 섹시한 향 (주요 성분: 아니스, 아사페티다, 카다멈, 계피, 정향 잎, 큐민, 딜, 생강, 샤프란)
시프레 (Cypre): 격조 있고 성숙한 느낌의 개성 있는 향 (주요 성분: 베르가못, 라브다눔)
과일향 (Fruity): 달콤하고 귀여운 향 (주요 성분: 바나나, 블랙베리, 체리, 망고, 딸기, 사과, 자두)
시트러스 (Citrus): 과일 같은 상큼한 향 (주요 성분: 베르가못, 자몽, 레몬, 유자, 감귤)
녹색향 (Green): 상쾌하고 신선한 향 (주요 성분: 오이, 민트, 티, 토마토 잎, 잔디)
알데하이드 (Aldehyde): 인공적인 비누향, 모던하고 우아하며 지적인 느낌 (주요 성분: 알데하이드, 코라놀, 에틸 말톨)
아쿠아틱 (Aquatic): 물과 바다의 시원함이 가득한 향 (주요 성분: 아쿠오존, 캘론, 해수, 물방울)
푸제르 (Fougere): 부드럽고 싱그러운 남성적인 향 (주요 성분: 양치식물, 라벤더, 쿠마린, 스윗그라스)
구르망 (Gourmand): 달콤하고 유혹적인 미각의 즐거움 (주요 성분: 아몬드, 버터, 초콜릿, 아이스크림, 마카롱, 바닐라)
우디 (Woody): 따뜻하고 부드러운 고급스러운 향 (주요 성분: 오드, 삼나무, 참나무, 샌달우드)
오리엔탈 (Oriental): 무거우면서 부드럽고 고혹적인 달콤한 향 (주요 성분: 바닐라, 벤조인, 몰약, 프랭킨센스)
플로럴 (Floral): 화려하고 여성스러운 향 (주요 성분: 라벤더, 자스민, 로즈, 모란)
머스크 (Musk): 이성에게 어필 가능한 아름다운 향 (주요 성분: 머스크, 앰브레트, 실콜라이드)
파우더리 (Powdery): 포근하고 부드러운 향 (주요 성분: 아이리스, 헬리오트로프, 오리스 루트)
담배/가죽향 (Tobacco Leather): 마초적인 남성의 강렬한 매력 (주요 성분: 가죽, 스웨이드, 담배 꽃)
향수: 오드 우드 오 드 퍼퓸, 매칭된 노트 개수: 0, 매칭된 노트: []
향수: 플레르 드 뽀 오 드